In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook
import re
from bs4 import BeautifulSoup
import os
import re
import gc
import sys
import time
import json
import random
import unicodedata
import multiprocessing
from functools import partial, lru_cache

import emoji

import numpy as np
import pandas as pd
from sklearn.externals import joblib
from tqdm import tqdm, tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

from nltk import TweetTokenizer
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
import html
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
import os
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import *

np.set_printoptions(suppress=True)
print(tf.__version__)

In [ ]:
GOOGLE_PATH = "../input/google-quest-challenge/"
STACK_PATH = "../input/stackexchange123/StackexchangeExtract/"

train = pd.read_csv(GOOGLE_PATH+"train.csv")
test = pd.read_csv(GOOGLE_PATH+"test.csv")
sample_submission = pd.read_csv(GOOGLE_PATH+"sample_submission.csv")

train.shape, test.shape

In [ ]:
target_columns = sample_submission.columns[1:].values
target_columns.shape

In [ ]:
def url_id_ex(url):
    try:
        ids = int(url.split("/")[-2])
    except:
        ids = int(url.split("/")[-1])
    return ids

# category_type
train["category_type"] = train["url"].apply(lambda x : x.split(".")[0].split("/")[-1])
test["category_type"] = test["url"].apply(lambda x : x.split(".")[0].split("/")[-1])

train["quser_id"] = train["question_user_page"].apply(lambda x: int(x.split("/")[-1]))
train["auser_id"] = train["answer_user_page"].apply(lambda x: int(x.split("/")[-1]))
train["url_id"] = train["url"].apply(url_id_ex)

test["quser_id"] = test["question_user_page"].apply(lambda x: int(x.split("/")[-1]))
test["auser_id"] = test["answer_user_page"].apply(lambda x: int(x.split("/")[-1]))
test["url_id"] = test["url"].apply(url_id_ex)

train.category_type.replace("programmers", "softwareengineering", inplace=True)
test.category_type.replace("programmers", "softwareengineering", inplace=True)

In [ ]:
def final_dataframe(files_path, df):
    listofdir = list(os.listdir(files_path))
    listofdir.remove('dataset-metadata.json')
    final_list = []
    
    posts_columns = None
    q_users_columns = None
    a_users_columns = None
    
    for file in tqdm_notebook(listofdir):
        temp_df = df[df.category_type == file]
        
        temp_users = pd.read_csv(STACK_PATH+file+"/user_df.csv")
        temp_posts = pd.read_csv(STACK_PATH+file+"/posts_df.csv")
        temp_users_columns = temp_users.columns.values
        
        posts_columns = temp_posts.columns.values
        temp_df = pd.merge(temp_df, temp_posts, left_on="url_id", right_on="Id", how="left")
        
        del temp_posts
        
        temp_users = temp_users.add_prefix("q_")
        q_users_columns = temp_users.columns.values
        temp_df = pd.merge(temp_df, temp_users, left_on="quser_id", right_on="q_Id", how="left")
        
        temp_users.columns = temp_users_columns
        
        temp_users = temp_users.add_prefix("a_")
        a_users_columns = temp_users.columns.values
        temp_df = pd.merge(temp_df, temp_users, left_on="auser_id", right_on="a_Id", how="left")
        del temp_users
        
        temp_df = temp_df.to_dict("records")
        final_list.extend(temp_df)
        del temp_df
        
    total_columns_dic = {
        "posts_columns":posts_columns,
        "q_users_columns":q_users_columns,
        "a_users_columns":a_users_columns
    }
    
    final_df = pd.DataFrame(final_list)
    del final_list
        
    return final_df, total_columns_dic

In [ ]:
!cat /proc/meminfo | grep Mem

In [ ]:
%%time
files_path = STACK_PATH
train_final, total_columns_dic = final_dataframe(files_path, train)
test_final, total_columns_dic = final_dataframe(files_path, test)

In [ ]:
train_final.shape, test_final.shape

In [ ]:
stackof_train = train[train.category == "STACKOVERFLOW"].copy()
stackof_test = test[test.category == "STACKOVERFLOW"].copy()
stackof_train.shape , stackof_test.shape

In [ ]:
train_final = train_final.append(stackof_train)
test_final = test_final.append(stackof_test)
train_final.shape, test_final.shape

In [ ]:
test_columns = test.columns.values.tolist()
train_columns = train.columns.values.tolist()
train_final = pd.merge(train, train_final, left_on=train_columns, right_on=train_columns, how="left")
test_final = pd.merge(test, test_final, left_on=test_columns, right_on=test_columns, how="left")

In [ ]:
train_final.shape , test_final.shape

In [ ]:
!cat /proc/meminfo | grep Mem

In [ ]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    """

    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))

    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            #df[col] = df[col].astype("category")
            pass
        
    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df

from bs4 import BeautifulSoup
def removing_html_tags(raw_html):
    cleantext = BeautifulSoup(raw_html, "lxml").text
    return cleantext

def replace_urls(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', ' web ', text, flags=re.MULTILINE)
    return text

def clean_AboutMe(text):
    text = removing_html_tags(text)
    text = replace_urls(text)
    return text

def clean_Name(text):
    text = str(text)
    text = re.sub(r' ', '_', text, flags=re.MULTILINE)
    text = re.sub(r',',' ', text, flags=re.MULTILINE)
    return text

def clean_Class(text):
    text = str(text)
    text = re.sub(r',', ' ',text, flags=re.MULTILINE)
    return text

def log_transform_apply(value):
    if value == 0:
        return value
    elif value < 1:
        value = np.log1p(abs(value))
        return -value
    else:
        return np.log1p(value)

In [ ]:
train_final_targets = train_final[target_columns].copy()
train_final_targets["qa_id"] = train_final["qa_id"]
train_final.drop(columns=target_columns, inplace=True)

In [ ]:
train_final = reduce_mem_usage(train_final)
test_final = reduce_mem_usage(test_final)

In [ ]:
# User AboutMe
train_final["q_AboutMe_nan"] = train_final["q_AboutMe"].isnull().astype(int)
train_final["a_AboutMe_nan"] = train_final["a_AboutMe"].isnull().astype(int)   

test_final["q_AboutMe_nan"] = test_final["q_AboutMe"].isnull().astype(int)
test_final["a_AboutMe_nan"] = test_final["a_AboutMe"].isnull().astype(int) 

train_final["q_AboutMe"].fillna("nane", inplace=True)
train_final["a_AboutMe"].fillna("nane", inplace=True)

test_final["q_AboutMe"].fillna("nane", inplace=True)
test_final["a_AboutMe"].fillna("nane", inplace=True)

train_final["q_AboutMe"] = train_final["q_AboutMe"].apply(clean_AboutMe)
train_final["a_AboutMe"] = train_final["a_AboutMe"].apply(clean_AboutMe)

test_final["q_AboutMe"] = test_final["q_AboutMe"].apply(clean_AboutMe)
test_final["a_AboutMe"] = test_final["a_AboutMe"].apply(clean_AboutMe)

# User TagBased

train_final["q_TagBased_nan"] = train_final["q_TagBased"].isnull().astype(int)
train_final["a_TagBased_nan"] = train_final["a_TagBased"].isnull().astype(int)   

test_final["q_TagBased_nan"] = test_final["q_TagBased"].isnull().astype(int)
test_final["a_TagBased_nan"] = test_final["a_TagBased"].isnull().astype(int) 


train_final["q_TagBased"].fillna("nane", inplace=True)
train_final["a_TagBased"].fillna("nane", inplace=True)

test_final["q_TagBased"].fillna("nane", inplace=True)
test_final["a_TagBased"].fillna("nane", inplace=True)

train_final["q_TagBased"] = train_final["q_TagBased"].apply(clean_Class)
train_final["a_TagBased"] = train_final["a_TagBased"].apply(clean_Class)

test_final["q_TagBased"] = test_final["q_TagBased"].apply(clean_Class)
test_final["a_TagBased"] = test_final["a_TagBased"].apply(clean_Class)

# User Name

train_final["q_Name_nan"] = train_final["q_Name"].isnull().astype(int)
train_final["a_Name_nan"] = train_final["a_Name"].isnull().astype(int)   

test_final["q_Name_nan"] = test_final["q_Name"].isnull().astype(int)
test_final["a_Name_nan"] = test_final["a_Name"].isnull().astype(int) 


train_final["q_Name"].fillna("nane", inplace=True)
train_final["a_Name"].fillna("nane", inplace=True)

test_final["q_Name"].fillna("nane", inplace=True)
test_final["a_Name"].fillna("nane", inplace=True)

train_final["q_Name"] = train_final["q_Name"].apply(clean_Name)
train_final["a_Name"] = train_final["a_Name"].apply(clean_Name)

test_final["q_Name"] = test_final["q_Name"].apply(clean_Name)
test_final["a_Name"] = test_final["a_Name"].apply(clean_Name)

# User Class

train_final["q_Class"].fillna("0", inplace=True)
train_final["a_Class"].fillna("0", inplace=True)

test_final["q_Class"].fillna("0", inplace=True)
test_final["a_Class"].fillna("0", inplace=True)

train_final["q_Class"] = train_final["q_Class"].apply(clean_Class)
train_final["a_Class"] = train_final["a_Class"].apply(clean_Class)

test_final["q_Class"] = test_final["q_Class"].apply(clean_Class)
test_final["a_Class"] = test_final["a_Class"].apply(clean_Class)

# User Views
train_final["q_Views_nan"] = train_final["q_Views"].isnull().astype(int)
train_final["a_Views_nan"] = train_final["a_Views"].isnull().astype(int)   

test_final["q_Views_nan"] = test_final["q_Views"].isnull().astype(int)
test_final["a_Views_nan"] = test_final["a_Views"].isnull().astype(int) 

train_final["q_Views"].fillna(0, inplace=True)
train_final["a_Views"].fillna(0, inplace=True)

test_final["q_Views"].fillna(0, inplace=True)
test_final["a_Views"].fillna(0, inplace=True)

# User UpVotes
#train_final["q_Views_nan"] = train_final["q_Views"].isnull().astype(int)
#train_final["a_Views_nan"] = train_final["a_Views"].isnull().astype(int)   

#test_final["q_Views_nan"] = test_final["q_Views"].isnull().astype(int)
#test_final["a_Views_nan"] = test_final["a_Views"].isnull().astype(int)

# User UpVotes

train_final["q_UpVotes_nan"] = train_final["q_UpVotes"].isnull().astype(int)
train_final["a_UpVotes_nan"] = train_final["a_UpVotes"].isnull().astype(int)   

test_final["q_UpVotes_nan"] = test_final["q_UpVotes"].isnull().astype(int)
test_final["a_UpVotes_nan"] = test_final["a_UpVotes"].isnull().astype(int) 

train_final["q_UpVotes"].fillna(0, inplace=True)
train_final["a_UpVotes"].fillna(0, inplace=True)

test_final["q_UpVotes"].fillna(0, inplace=True)
test_final["a_UpVotes"].fillna(0, inplace=True)

# User DownVotes
train_final["q_DownVotes_nan"] = train_final["q_DownVotes"].isnull().astype(int)
train_final["a_DownVotes_nan"] = train_final["a_DownVotes"].isnull().astype(int)   

test_final["q_DownVotes_nan"] = test_final["q_DownVotes"].isnull().astype(int)
test_final["a_DownVotes_nan"] = test_final["a_DownVotes"].isnull().astype(int) 

train_final["q_DownVotes"].fillna(0, inplace=True)
train_final["a_DownVotes"].fillna(0, inplace=True)

test_final["q_DownVotes"].fillna(0, inplace=True)
test_final["a_DownVotes"].fillna(0, inplace=True)

user_drop_cols = ["q_Id", "q_DisplayName", "q_UserId", "a_Id", "a_DisplayName", "a_UserId"]

In [ ]:
def clean_Tags(text):
    text = str(text)
    text = re.sub(r'><', '> <', text, flags=re.MULTILINE)
    text = re.sub(r'>', '', text, flags=re.MULTILINE)
    text = re.sub(r'<', '', text, flags=re.MULTILINE)
    text = ''.join([i for i in text if not i.isdigit()])
    return text

In [ ]:
# Posts PostTypeId
train_final["PostTypeId_nan"] = train_final["PostTypeId"].isnull().astype(int)
test_final["PostTypeId_nan"] = test_final["PostTypeId"].isnull().astype(int) 

train_final["PostTypeId"].fillna(1.0, inplace=True)
test_final["PostTypeId"].fillna(1.0, inplace=True)

# Posts Score
train_final["Score_nan"] = train_final["Score"].isnull().astype(int)
test_final["Score_nan"] = test_final["Score"].isnull().astype(int) 

train_final["Score"].fillna(0, inplace=True)
test_final["Score"].fillna(0, inplace=True)

train_final["Score"] = train_final["Score"].apply(log_transform_apply)
test_final["Score"] = test_final["Score"].apply(log_transform_apply)

# Posts ViewCount
train_final["ViewCount_nan"] = train_final["ViewCount"].isnull().astype(int)
test_final["ViewCount_nan"] = test_final["ViewCount"].isnull().astype(int) 

train_final["ViewCount"].fillna(0, inplace=True)
test_final["ViewCount"].fillna(0, inplace=True)

train_final["ViewCount"] = np.log1p(abs(train_final["ViewCount"]))
test_final["ViewCount"] = np.log1p(abs(test_final["ViewCount"]))

# Posts Tags
train_final["Tags_nan"] = train_final["Tags"].isnull().astype(int)
test_final["Tags_nan"] = test_final["Tags"].isnull().astype(int) 

train_final["Tags"].fillna("<nanetag>", inplace=True)
test_final["Tags"].fillna("<nanetag>", inplace=True)

train_final["Tags"] = train_final["Tags"].apply(clean_Tags)
test_final["Tags"] = test_final["Tags"].apply(clean_Tags)


# Posts AnswerCount
train_final["AnswerCount_nan"] = train_final["AnswerCount"].isnull().astype(int)
test_final["AnswerCount_nan"] = test_final["AnswerCount"].isnull().astype(int) 

train_final["AnswerCount"].fillna(1, inplace=True)
test_final["AnswerCount"].fillna(1, inplace=True)


# Posts CommentCount
train_final["CommentCount_nan"] = train_final["CommentCount"].isnull().astype(int)
test_final["CommentCount_nan"] = test_final["CommentCount"].isnull().astype(int) 

train_final["CommentCount"].fillna(0, inplace=True)
test_final["CommentCount"].fillna(0, inplace=True)

# Posts FavoriteCount
train_final["FavoriteCount_nan"] = train_final["FavoriteCount"].isnull().astype(int)
test_final["FavoriteCount_nan"] = test_final["FavoriteCount"].isnull().astype(int) 

train_final["FavoriteCount"].fillna(0, inplace=True)
test_final["FavoriteCount"].fillna(0, inplace=True)

train_final["FavoriteCount"] = train_final["FavoriteCount"].apply(log_transform_apply)
test_final["FavoriteCount"] = test_final["FavoriteCount"].apply(log_transform_apply)


posts_drop_cols = ['Id', 'AcceptedAnswerId', 'OwnerUserId', 'LastActivityDate', 'ParentId', 'ClosedDate', 'LastEditorDisplayName', 'OwnerDisplayName', 'CommunityOwnedDate']

In [ ]:
def get_code_html(text, body):
    if text == np.nan:
        body = str(body)
        code_list = []
        codes_list1 = re.findall(':\n\n.*?\n\n\n',body, flags=re.DOTALL)
        codes_list2 = re.findall('.\n\n(.*?)\n\n\n',body, flags=re.DOTALL)
        codes_list3 = re.findall('{(.*?)}',body, flags=re.DOTALL)
        code_list.extend(codes_list1)
        code_list.extend(codes_list2)
        code_list.extend(codes_list3)
        if len(codes_list) > 0:
            code = '<#next#>'.join(map(str, codes_list))
            return code
        else:
            return "NONE"
    else:
        text = str(text)
        codes_list = re.findall('<code>(.*?)</code>',text, flags=re.DOTALL)
        if len(codes_list) > 0:
            code = '<#next#>'.join(map(str, codes_list))
            return code
        else:
            return "NONE"
    
def get_code_replace(text, code):
    text = str(text)
    code = str(code)
    if code != "NONE":
        codes_list = code.split("<#next#>")
        codes_list = sorted(codes_list, key=len,reverse=True)
        for i in codes_list:
            i = re.escape(i)
            text = re.sub(f"{i}", ' [CODE] ', text, flags=re.DOTALL)
        return text
    else:
        return text      
    
def get_blockquote_html(text):
    text = str(text)
    codes_list = re.findall('<blockquote>(.*?)</blockquote>',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        code = ' '.join(map(str, codes_list))
        return code
    else:
        return "NONE"
    
def get_slsldolel(text):
    text = str(text)
    codes_list = re.findall('\\\\\$(.*?)\\\\\$',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        code = ' '.join(map(str, codes_list))
        return code
    else:
        return "NONE"

def get_slsldolel_replace(text):
    text = str(text)
    codes_list = re.findall('\\\\\$.*?\\\\\$',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        #code = ' '.join(map(str, codes_list))
        for i in codes_list:
            i = re.escape(i)
            text = re.sub(f"{i}", ' [FORMULA] ', text)
        return text
    else:
        return text

def get_doldol(text):
    text = str(text)
    codes_list = re.findall('\$\$(.*?)\$\$',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        code = ' '.join(map(str, codes_list))
        return code
    else:
        return "NONE"

def get_doldol_replace(text):
    text = str(text)
    codes_list = re.findall('\$\$.*?\$\$',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        #code = ' '.join(map(str, codes_list))
        for i in codes_list:
            i = re.escape(i)
            text = re.sub(f"{i}", ' [FORMULA] ', text)
        return text
    else:
        return text

def get_spdol(text):
    text = str(text)
    codes_list = re.findall(' \$(.*?) \$',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        code = ' '.join(map(str, codes_list))
        return code
    else:
        return "NONE"
    
def get_spdol_replace(text):
    text = str(text)
    codes_list = re.findall(' \$.*? \$',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        #code = ' '.join(map(str, codes_list))
        for i in codes_list:
            i = re.escape(i)
            text = re.sub(f"{i}", ' [FORMULA] ', text)
        return text
    else:
        return text

def get_dol(text):
    text = str(text)
    codes_list = re.findall('\$(.*?)\$',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        code = ' '.join(map(str, codes_list))
        return code
    else:
        return "NONE"
    
def get_dol_replace(text):
    text = str(text)
    codes_list = re.findall('\$.*?\$',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        #code = ' '.join(map(str, codes_list))
        for i in codes_list:
            i = re.escape(i)
            text = re.sub(f"{i}", ' [FORMULA] ', text)
        return text
    else:
        return text

def get_code1(text):
    text = str(text)
    codes_list = re.findall(':\n\n(.*?)\n\n\n',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        code = ' . '.join(map(str, codes_list))
        return code
    else:
        return "NONE"

def get_code1_replace(text):
    text = str(text)
    codes_list = re.findall(':\n\n.*?\n\n\n',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        #code = ' '.join(map(str, codes_list))
        for i in codes_list:
            i = re.escape(i)
            text = re.sub(f"{i}", ' [CODE] ', text)
        return text
    else:
        return text

def get_code2(text):
    text = str(text)
    codes_list = re.findall('.\n\n(.*?)\n\n\n',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        code = ' . '.join(map(str, codes_list))
        return code
    else:
        return "NONE"

def get_code2_replace(text):
    text = str(text)
    codes_list = re.findall('.\n\n.*?\n\n\n',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        #code = ' '.join(map(str, codes_list))
        for i in codes_list:
            if i.count("\n") > 4:
                i = re.escape(i)
                text = re.sub(f"{i}", ' [CODE] ', text)
            else:
                pass
        return text
    else:
        return text
    
def get_code3(text):
    text = str(text)
    codes_list = re.findall('{(.*?)}',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        code = ' . '.join(map(str, codes_list))
        return code
    else:
        return "NONE"

def get_code3_replace(text):
    text = str(text)
    codes_list = re.findall('{.*?}',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        #code = ' '.join(map(str, codes_list))
        for i in codes_list:
            if len(i) > 10:
                i = re.escape(i)
                text = re.sub(f"{i}", ' [CODE] ', text)
            else:
                pass
        return text
    else:
        return text

In [ ]:
# question_body_code
train_final["question_body_code"] = train_final.apply(lambda x: get_code_html(x["Body"], x["question_body"]), axis=1)
train_final["question_body_clean"] = train_final.apply(lambda x: get_code_replace(x["question_body"], x["question_body_code"]), axis=1)

test_final["question_body_code"] = test_final.apply(lambda x: get_code_html(x["Body"], x["question_body"]), axis=1)
test_final["question_body_clean"] = test_final.apply(lambda x: get_code_replace(x["question_body"], x["question_body_code"]), axis=1)

# question_body_slsldolel
train_final["question_body_slsldolel"] = train_final["question_body"].apply(get_slsldolel)
train_final["question_body_clean"] = train_final["question_body_clean"].apply(get_slsldolel_replace)

test_final["question_body_slsldolel"] = test_final["question_body"].apply(get_slsldolel)
test_final["question_body_clean"] = test_final["question_body_clean"].apply(get_slsldolel_replace)

# question_body_doldol
train_final["question_body_doldol"] = train_final["question_body"].apply(get_doldol)
train_final["question_body_clean"] = train_final["question_body_clean"].apply(get_doldol_replace)

test_final["question_body_doldol"] = test_final["question_body"].apply(get_doldol)
test_final["question_body_clean"] = test_final["question_body_clean"].apply(get_doldol_replace)

# question_body_spdol
train_final["question_body_spdol"] = train_final["question_body"].apply(get_spdol)
train_final["question_body_clean"] = train_final["question_body_clean"].apply(get_spdol_replace)

test_final["question_body_spdol"] = test_final["question_body"].apply(get_spdol)
test_final["question_body_clean"] = test_final["question_body_clean"].apply(get_spdol_replace)

# question_body_dol
train_final["question_body_dol"] = train_final["question_body"].apply(get_dol)
train_final["question_body_clean"] = train_final["question_body_clean"].apply(get_dol_replace)

test_final["question_body_dol"] = test_final["question_body"].apply(get_dol)
test_final["question_body_clean"] = test_final["question_body_clean"].apply(get_dol_replace)

train_final["question_body_all"] = list(map(lambda a,b,c,d,e : str(a) + ' ' + str(b) + ' ' + str(c) +' '+ str(d) +' '+ str(e),train_final["question_body_code"],train_final["question_body_slsldolel"],train_final["question_body_doldol"],train_final["question_body_spdol"], train_final["question_body_dol"]))
test_final["question_body_all"] = list(map(lambda a,b,c,d,e : str(a) + ' ' + str(b) + ' ' + str(c) +' '+ str(d) +' '+ str(e),test_final["question_body_code"],test_final["question_body_slsldolel"],test_final["question_body_doldol"],test_final["question_body_spdol"], test_final["question_body_dol"]))

In [ ]:
def get_code1(text):
    text = str(text)
    codes_list = re.findall(':\n\n(.*?)\n\n\n',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        code = ' . '.join(map(str, codes_list))
        return code
    else:
        return "NONE"

def get_code1_replace(text):
    text = str(text)
    codes_list = re.findall(':\n\n.*?\n\n\n',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        #code = ' '.join(map(str, codes_list))
        for i in codes_list:
            i = re.escape(i)
            text = re.sub(f"{i}", ' [CODE] ', text)
        return text
    else:
        return text

def get_code2(text):
    text = str(text)
    codes_list = re.findall('.\n\n(.*?)\n\n\n',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        code = ' . '.join(map(str, codes_list))
        return code
    else:
        return "NONE"

def get_code2_replace(text):
    text = str(text)
    codes_list = re.findall('.\n\n.*?\n\n\n',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        #code = ' '.join(map(str, codes_list))
        for i in codes_list:
            if i.count("\n") > 4:
                i = re.escape(i)
                text = re.sub(f"{i}", ' [CODE] ', text)
            else:
                pass
        return text
    else:
        return text
    
def get_code3(text):
    text = str(text)
    codes_list = re.findall('{(.*?)}',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        code = ' . '.join(map(str, codes_list))
        return code
    else:
        return "NONE"

def get_code3_replace(text):
    text = str(text)
    codes_list = re.findall('{.*?}',text, flags=re.DOTALL)
    if len(codes_list) > 0:
        #code = ' '.join(map(str, codes_list))
        for i in codes_list:
            if len(i) > 10:
                i = re.escape(i)
                text = re.sub(f"{i}", ' [CODE] ', text)
            else:
                pass
        return text
    else:
        return text

In [ ]:
# answer_code1

train_final["answer_code1"] = train_final["answer"].apply(get_code1)
train_final["answer_clean"] = train_final["answer"].apply(get_code1_replace)

test_final["answer_code1"] = test_final["answer"].apply(get_code1)
test_final["answer_clean"] = test_final["answer"].apply(get_code1_replace)

# answer_code2

train_final["answer_code2"] = train_final["answer"].apply(get_code2)
train_final["answer_clean"] = train_final["answer_clean"].apply(get_code2_replace)

test_final["answer_code2"] = test_final["answer"].apply(get_code2)
test_final["answer_clean"] = test_final["answer_clean"].apply(get_code2_replace)

# answer_code3

train_final["answer_code3"] = train_final["answer"].apply(get_code3)
train_final["answer_clean"] = train_final["answer_clean"].apply(get_code3_replace)

test_final["answer_code3"] = test_final["answer"].apply(get_code3)
test_final["answer_clean"] = test_final["answer_clean"].apply(get_code3_replace)

train_final["answer_code"] = list(map(lambda a,b,c : str(a) + ' ' + str(b) + ' ' + str(c),train_final["answer_code1"],train_final["answer_code2"],train_final["answer_code3"]))
test_final["answer_code"] = list(map(lambda a,b,c : str(a) + ' ' + str(b) + ' ' + str(c),test_final["answer_code1"],test_final["answer_code2"],test_final["answer_code3"]))

# question_body_slsldolel
train_final["answer_slsldolel"] = train_final["answer"].apply(get_slsldolel)
train_final["answer_clean"] = train_final["answer_clean"].apply(get_slsldolel_replace)

test_final["answer_slsldolel"] = test_final["answer"].apply(get_slsldolel)
test_final["answer_clean"] = test_final["answer_clean"].apply(get_slsldolel_replace)

# question_body_doldol
train_final["answer_doldol"] = train_final["answer"].apply(get_doldol)
train_final["answer_clean"] = train_final["answer_clean"].apply(get_doldol_replace)

test_final["answer_doldol"] = test_final["answer"].apply(get_doldol)
test_final["answer_clean"] = test_final["answer_clean"].apply(get_doldol_replace)

# question_body_spdol
train_final["answer_spdol"] = train_final["answer"].apply(get_spdol)
train_final["answer_clean"] = train_final["answer_clean"].apply(get_spdol_replace)

test_final["answer_spdol"] = test_final["answer"].apply(get_spdol)
test_final["answer_clean"] = test_final["answer_clean"].apply(get_spdol_replace)

# question_body_dol
train_final["answer_dol"] = train_final["answer"].apply(get_dol)
train_final["answer_clean"] = train_final["answer_clean"].apply(get_dol_replace)

test_final["answer_dol"] = test_final["answer"].apply(get_dol)
test_final["answer_clean"] = test_final["answer_clean"].apply(get_dol_replace)

train_final["answer_all"] = list(map(lambda a,b,c,d,e : str(a) + ' ' + str(b) + ' ' + str(c) +' '+ str(d)+' '+ str(e),train_final["answer_code"],train_final["answer_slsldolel"],train_final["answer_doldol"],train_final["answer_spdol"],train_final["answer_dol"]))
test_final["answer_all"] = list(map(lambda a,b,c,d,e : str(a) + ' ' + str(b) + ' ' + str(c) +' '+ str(d)+' '+ str(e),test_final["answer_code"],test_final["answer_slsldolel"],test_final["answer_doldol"],test_final["answer_spdol"], test_final["answer_dol"]))

In [ ]:
droped_columns = []
droped_columns.extend(user_drop_cols)
droped_columns.extend(posts_drop_cols)
print(len(droped_columns))

train_final.drop(columns=droped_columns, inplace=True)
test_final.drop(columns=droped_columns, inplace=True)

In [ ]:
train_final.shape, test_final.shape, train_final_targets.shape

## Text cleaning

In [ ]:
def url_replace(text):
    text = re.sub("(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+", '[URL]' ,text, flags=re.DOTALL)
    return text

def url_count(text):
    count = len(re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+',text))
    return count

In [ ]:
CUSTOM_TABLE = str.maketrans(
    {
        "\xad": None,
        "\x7f": None,
        "\ufeff": None,
        "\u200b": None,
        "\u200e": None,
        "\u202a": None,
        "\u202c": None,
        "‘": "'",
        "’": "'",
        "`": "'",
        "“": '"',
        "”": '"',
        "«": '"',
        "»": '"',
        "ɢ": "G",
        "ɪ": "I",
        "ɴ": "N",
        "ʀ": "R",
        "ʏ": "Y",
        "ʙ": "B",
        "ʜ": "H",
        "ʟ": "L",
        "ғ": "F",
        "ᴀ": "A",
        "ᴄ": "C",
        "ᴅ": "D",
        "ᴇ": "E",
        "ᴊ": "J",
        "ᴋ": "K",
        "ᴍ": "M",
        "Μ": "M",
        "ᴏ": "O",
        "ᴘ": "P",
        "ᴛ": "T",
        "ᴜ": "U",
        "ᴡ": "W",
        "ᴠ": "V",
        "ĸ": "K",
        "в": "B",
        "м": "M",
        "н": "H",
        "т": "T",
        "ѕ": "S",
        "—": "-",
        "–": "-",
    }
)

WORDS_REPLACER = [
    ("sh*t", "shit"),
    ("s**t", "shit"),
    ("f*ck", "fuck"),
    ("fu*k", "fuck"),
    ("f**k", "fuck"),
    ("f*****g", "fucking"),
    ("f***ing", "fucking"),
    ("f**king", "fucking"),
    ("p*ssy", "pussy"),
    ("p***y", "pussy"),
    ("pu**y", "pussy"),
    ("p*ss", "piss"),
    ("b*tch", "bitch"),
    ("bit*h", "bitch"),
    ("h*ll", "hell"),
    ("h**l", "hell"),
    ("cr*p", "crap"),
    ("d*mn", "damn"),
    ("stu*pid", "stupid"),
    ("st*pid", "stupid"),
    ("n*gger", "nigger"),
    ("n***ga", "nigger"),
    ("f*ggot", "faggot"),
    ("scr*w", "screw"),
    ("pr*ck", "prick"),
    ("g*d", "god"),
    ("s*x", "sex"),
    ("a*s", "ass"),
    ("a**hole", "asshole"),
    ("a***ole", "asshole"),
    ("a**", "ass"),
]

WORDS_REPLACER_2 = [
    ("ain't", 'is not'),
     ("aren't", 'are not'),
     ("can't", 'cannot'),
     ("'cause", 'because'),
     ("could've", 'could have'),
     ("couldn't", 'could not'),
     ("didn't", 'did not'),
     ("doesn't", 'does not'),
     ("don't", 'do not'),
     ("hadn't", 'had not'),
     ("hasn't", 'has not'),
     ("haven't", 'have not'),
     ("he'd", 'he would'),
     ("he'll", 'he will'),
     ("he's", 'he is'),
     ("how'd", 'how did'),
     ("how'd'y", 'how do you'),
     ("how'll", 'how will'),
     ("how's", 'how is'),
     ("i'd", 'i would'),
     ("i'd've", 'i would have'),
     ("i'll", 'i will'),
     ("i'll've", 'i will have'),
     ("i'm", 'i am'),
     ("i've", 'i have'),
     ("i'd", 'i would'),
     ("i'd've", 'i would have'),
     ("i'll", 'i will'),
     ("i'll've", 'i will have'),
     ("i'm", 'i am'),
     ("i've", 'i have'),
     ("isn't", 'is not'),
     ("it'd", 'it would'),
     ("it'd've", 'it would have'),
     ("it'll", 'it will'),
     ("it'll've", 'it will have'),
     ("it's", 'it is'),
     ("let's", 'let us'),
     ("ma'am", 'madam'),
     ("mayn't", 'may not'),
     ("might've", 'might have'),
     ("mightn't", 'might not'),
     ("mightn't've", 'might not have'),
     ("must've", 'must have'),
     ("mustn't", 'must not'),
     ("mustn't've", 'must not have'),
     ("needn't", 'need not'),
     ("needn't've", 'need not have'),
     ("o'clock", 'of the clock'),
     ("oughtn't", 'ought not'),
     ("oughtn't've", 'ought not have'),
     ("shan't", 'shall not'),
     ("sha'n't", 'shall not'),
     ("shan't've", 'shall not have'),
     ("she'd", 'she would'),
     ("she'd've", 'she would have'),
     ("she'll", 'she will'),
     ("she'll've", 'she will have'),
     ("she's", 'she is'),
     ("should've", 'should have'),
     ("shouldn't", 'should not'),
     ("shouldn't've", 'should not have'),
     ("so've", 'so have'),
     ("so's", 'so as'),
     ("this's", 'this is'),
     ("that'd", 'that would'),
     ("that'd've", 'that would have'),
     ("that's", 'that is'),
     ("there'd", 'there would'),
     ("there'd've", 'there would have'),
     ("there's", 'there is'),
     ("here's", 'here is'),
     ("they'd", 'they would'),
     ("they'd've", 'they would have'),
     ("they'll", 'they will'),
     ("they'll've", 'they will have'),
     ("they're", 'they are'),
     ("they've", 'they have'),
     ("to've", 'to have'),
     ("wasn't", 'was not'),
     ("we'd", 'we would'),
     ("we'd've", 'we would have'),
     ("we'll", 'we will'),
     ("we'll've", 'we will have'),
     ("we're", 'we are'),
     ("we've", 'we have'),
     ("weren't", 'were not'),
     ("what'll", 'what will'),
     ("what'll've", 'what will have'),
     ("what're", 'what are'),
     ("what's", 'what is'),
     ("what've", 'what have'),
     ("when's", 'when is'),
     ("when've", 'when have'),
     ("where'd", 'where did'),
     ("where's", 'where is'),
     ("where've", 'where have'),
     ("who'll", 'who will'),
     ("who'll've", 'who will have'),
     ("who's", 'who is'),
     ("who've", 'who have'),
     ("why's", 'why is'),
     ("why've", 'why have'),
     ("will've", 'will have'),
     ("won't", 'will not'),
     ("won't've", 'will not have'),
     ("would've", 'would have'),
     ("wouldn't", 'would not'),
     ("wouldn't've", 'would not have'),
     ("y'all", 'you all'),
     ("y'all'd", 'you all would'),
     ("y'all'd've", 'you all would have'),
     ("y'all're", 'you all are'),
     ("y'all've", 'you all have'),
     ("you'd", 'you would'),
     ("you'd've", 'you would have'),
     ("you'll", 'you will'),
     ("you'll've", 'you will have'),
     ("you're", 'you are'),
     ("you've", 'you have'),
     ('what”s', 'what is'),
     ('what"s', 'what is'),
     ('its', 'it is'),
     ("what's", 'what is'),
     ("'ll", 'will'),
     ("n't", 'not'),
     ("'re", 'are'),
     ("ain't", 'is not'),
     ("aren't", 'are not'),
     ("can't", 'cannot'),
     ("'cause", 'because'),
     ("could've", 'could have'),
     ("couldn't", 'could not'),
     ("didn't", 'did not'),
     ("doesn't", 'does not'),
     ("don't", 'do not'),
     ("hadn't", 'had not'),
     ("hasn't", 'has not'),
     ("haven't", 'have not'),
     ("he'd", 'he would'),
     ("he'll", 'he will'),
     ("he's", 'he is'),
     ("how'd", 'how did'),
     ("how'd'y", 'how do you'),
     ("how'll", 'how will'),
     ("how's", 'how is'),
     ("i'd", 'i would'),
     ("i'd've", 'i would have'),
     ("i'll", 'i will'),
     ("i'll've", 'i will have'),
     ("i'm", 'i am'),
     ("i've", 'i have'),
     ("i'd", 'i would'),
     ("i'd've", 'i would have'),
     ("i'll", 'i will'),
     ("i'll've", 'i will have'),
     ("i'm", 'i am'),
     ("i've", 'i have'),
     ("isn't", 'is not'),
     ("it'd", 'it would'),
     ("it'd've", 'it would have'),
     ("it'll", 'it will'),
     ("it'll've", 'it will have'),
     ("it's", 'it is'),
     ("let's", 'let us'),
     ("ma'am", 'madam'),
     ("mayn't", 'may not'),
     ("might've", 'might have'),
     ("mightn't", 'might not'),
     ("mightn't've", 'might not have'),
     ("must've", 'must have'),
     ("mustn't", 'must not'),
     ("mustn't've", 'must not have'),
     ("needn't", 'need not'),
     ("needn't've", 'need not have'),
     ("o'clock", 'of the clock'),
     ("oughtn't", 'ought not'),
     ("oughtn't've", 'ought not have'),
     ("shan't", 'shall not'),
     ("sha'n't", 'shall not'),
     ("shan't've", 'shall not have'),
     ("she'd", 'she would'),
     ("she'd've", 'she would have'),
     ("she'll", 'she will'),
     ("she'll've", 'she will have'),
     ("she's", 'she is'),
     ("should've", 'should have'),
     ("shouldn't", 'should not'),
     ("shouldn't've", 'should not have'),
     ("so've", 'so have'),
     ("so's", 'so as'),
     ("this's", 'this is'),
     ("that'd", 'that would'),
     ("that'd've", 'that would have'),
     ("that's", 'that is'),
     ("there'd", 'there would'),
     ("there'd've", 'there would have'),
     ("there's", 'there is'),
     ("here's", 'here is'),
     ("they'd", 'they would'),
     ("they'd've", 'they would have'),
     ("'they're", 'they are'),
     ("they'll", 'they will'),
     ("they'll've", 'they will have'),
     ("they're", 'they are'),
     ("they've", 'they have'),
     ("to've", 'to have'),
     ("wasn't", 'was not'),
     ("we'd", 'we would'),
     ("we'd've", 'we would have'),
     ("we'll", 'we will'),
     ("we'll've", 'we will have'),
     ("we're", 'we are'),
     ("we've", 'we have'),
     ("weren't", 'were not'),
     ("what'll", 'what will'),
     ("what'll've", 'what will have'),
     ("what're", 'what are'),
     ("what's", 'what is'),
     ("what've", 'what have'),
     ("when's", 'when is'),
     ("when've", 'when have'),
     ("where'd", 'where did'),
     ("where's", 'where is'),
     ("where've", 'where have'),
     ("who'll", 'who will'),
     ("who'll've", 'who will have'),
     ("who's", 'who is'),
     ("who've", 'who have'),
     ("why's", 'why is'),
     ("why've", 'why have'),
     ("will've", 'will have'),
     ("won't", 'will not'),
     ("won't've", 'will not have'),
     ("would've", 'would have'),
     ("wouldn't", 'would not'),
     ("wouldn't've", 'would not have'),
     ("y'all", 'you all'),
     ("y'all'd", 'you all would'),
     ("y'all'd've", 'you all would have'),
     ("y'all're", 'you all are'),
     ("y'all've", 'you all have'),
     ("you'd", 'you would'),
     ("you'd've", 'you would have'),
     ("you'll", 'you will'),
     ("you'll've", 'you will have'),
     ("you're", 'you are'),
     ("you've", 'you have')
    ]

mispell_dict = {"aren't" : "are not",
                "can't" : "cannot",
                "couldn't" : "could not",
                "couldnt" : "could not",
                "didn't" : "did not",
                "doesn't" : "does not",
                "doesnt" : "does not",
                "don't" : "do not",
                "hadn't" : "had not",
                "hasn't" : "has not",
                "haven't" : "have not",
                "havent" : "have not",
                "he'd" : "he would",
                "he'll" : "he will",
                "he's" : "he is",
                "i'd" : "I would",
                "i'd" : "I had",
                "i'll" : "I will",
                "i'm" : "I am",
                "isn't" : "is not",
                "it's" : "it is",
                "it'll":"it will",
                "i've" : "I have",
                "let's" : "let us",
                "mightn't" : "might not",
                "mustn't" : "must not",
                "shan't" : "shall not",
                "she'd" : "she would",
                "she'll" : "she will",
                "she's" : "she is",
                "shouldn't" : "should not",
                "shouldnt" : "should not",
                "that's" : "that is",
                "thats" : "that is",
                "there's" : "there is",
                "theres" : "there is",
                "they'd" : "they would",
                "they'll" : "they will",
                "they're" : "they are",
                "theyre":  "they are",
                "they've" : "they have",
                "we'd" : "we would",
                "we're" : "we are",
                "weren't" : "were not",
                "we've" : "we have",
                "what'll" : "what will",
                "what're" : "what are",
                "what's" : "what is",
                "what've" : "what have",
                "where's" : "where is",
                "who'd" : "who would",
                "who'll" : "who will",
                "who're" : "who are",
                "who's" : "who is",
                "who've" : "who have",
                "won't" : "will not",
                "wouldn't" : "would not",
                "you'd" : "you would",
                "you'll" : "you will",
                "you're" : "you are",
                "you've" : "you have",
                "'re": " are",
                "wasn't": "was not",
                "we'll":" will",
                "didn't": "did not",
                "tryin'":"trying",
                "‘": "'", 
                "₹": "e", 
                "´": "'",
                "°": "", 
                "€": "e", 
                "™": "tm", 
                "√": " sqrt ",
                "×": "x", 
                "²": "2", 
                "—": "-",
                "–": "-",
                "’": "'",
                "_": "-",
                "`": "'", 
                '“': '"',
                '”': '"',
                '“': '"',
                "£": "e", 
                '∞': 'infinity',
                'θ': 'theta',
                '÷': '/',
                'α': 'alpha', 
                '•': '.',
                'à': 'a', 
                '−': '-', 
                'β': 'beta', 
                '∅': '', 
                '³': '3', 
                'π': 'pi',
                '\u200b': ' ', 
                '…': ' ... ', 
                '\ufeff': '', 
                'करना': '', 
                'है': '',  
                
               }


WORDS_REPLACER_3 = [(k, v) for k, v in mispell_dict.items()]

WORDS_REPLACER_4 = [('automattic', 'automatic'),
         ('sweetpotato', 'sweet potato'),
         ('statuscode', 'status code'),
         ('applylayer', 'apply layer'),
         ('aligator', 'alligator'),
         ('downloands', 'download'),
         ('dowloand', 'download'),
         ('thougths', 'thoughts'),
         ('helecopter', 'helicopter'),
         ('telugul', 'telugu'),
         ('unconditionaly', 'unconditionally'),
         ('coompanies', 'companies'),
         ('lndigenous', 'indigenous'),
         ('evluate', 'evaluate'),
         ('suggstion', 'suggestion'),
         ('thinkning', 'thinking'),
         ('concatinate', 'concatenate'),
         ('constitutionals', 'constitutional'),
         ('moneyback', 'money back'),
         ('civilazation', 'civilization'),
         ('paranoria', 'paranoia'),
         ('rightside', 'right side'),
         ('methamatics', 'mathematics'),
         ('natual', 'natural'),
         ('brodcast', 'broadcast'),
         ('pleasesuggest', 'please suggest'),
         ('intitution', 'institution'),
         ('experinces', 'experiences'),
         ('reallyreally', 'really'),
         ('testostreone', 'testosterone'),
         ('musceles', 'muscle'),
         ('bacause', 'because'),
         ('peradox', 'paradox'),
         ('probabity', 'probability'),
         ('collges', 'college'),
         ('diciplined', 'disciplined'),
         ('completeted', 'completed'),
         ('lunchshould', 'lunch should'),
         ('battlenet', 'battle net'),
         ('dissapoint', 'disappoint'),
         ('resultsnew', 'results new'),
         ('indcidents', 'incidents'),
         ('figuire', 'figure'),
         ('protonneutron', 'proton neutron'),
         ('tecnical', 'technical'),
         ('patern', 'pattern'),
         ('unenroll', 'un enroll'),
         ('proceedures', 'procedures'),
         ('srategy', 'strategy'),
         ('mordern', 'modern'),
         ('prepartion', 'preparation'),
         ('throuhout', 'throught'),
         ('academey', 'academic'),
         ('instituitions', 'institutions'),
         ('abadon', 'abandon'),
         ('compitetive', 'competitive'),
         ('hypercondriac', 'hypochondriac'),
         ('spiliting', 'splitting'),
         ('physchic', 'psychic'),
         ('flippingly', 'flipping'),
         ('likelyhood', 'likelihood'),
         ('armsindustry', 'arms industry'),
         (' turorials', 'tutorials'),
         ('photostats', 'photostat'),
         ('sunconcious', 'subconscious'),
         ('chemistryphysics', 'chemistry physics'),
         ('secondlife', 'second life'),
         ('histrorical', 'historical'),
         ('disordes', 'disorders'),
         ('differenturl', 'differential'),
         ('councilling', ' counselling'),
         ('sugarmill', 'sugar mill'),
         ('relatiosnhip', 'relationship'),
         ('fanpages', 'fan pages'),
         ('agregator', 'aggregator'),
         ('switc', 'switch'),
         ('smatphones', 'smartphones'),
         ('headsize', 'head size'),
         ('pendrives', 'pen drives'),
         ('biotecnology', 'biotechnology'),
         ('borderlink', 'border link'),
         ('furnance', 'furnace'),
         ('competetion', 'competition'),
         ('distibution', 'distribution'),
         ('ananlysis', ' analysis'),
         ('textile？', 'textile'),
         ('howww', 'how'),
         ('strategybusiness', 'strategy business'),
         ('spectrun', 'spectrum'),
         ('propasal', 'proposal'),
         ('appilcable', 'applicable'),
         ('accountwhat', ' account what'),
         ('algorithems', ' algorithms'),
         ('protuguese', ' Portuguese'),
         ('exatly', 'exactly'),
         ('disturbence', 'disturbance'),
         ('govrnment', 'government'),
         ('requiremnt', 'requirement'),
         ('vargin', 'virgin'),
         ('lonleley', 'lonely'),
         ('unmateralistic', 'materialistic'),
         ('dveloper', 'developer'),
         ('dcuments', 'documents'),
         ('techonologies', 'technologies'),
         ('morining', 'morning'),
         ('samsing', 'Samsung'),
         ('engeeniring', 'engineering'),
         ('racetrac', 'racetrack'),
         ('physian', 'physician'),
         ('theretell', 'there tell'),
         ('tryto', 'try to'),
         ('teamfight', 'team fight'),
         ('recomend', 'recommend'),
         ('spectables', 'spectacles'),
         ('emtional', 'emotional'),
         ('engeenerring', 'engineering'),
         ('optionsgood', 'options good'),
         ('primarykey', 'primary key'),
         ('foreignkey', 'foreign key'),
         ('concieved', 'conceived'),
         ('leastexpensive', 'least expensive'),
         ('foodtech', 'food tech'),
         ('electronegetivity', 'electronegativity'),
         ('polticians', 'politicians'),
         ('distruptive', 'disruptive'),
         ('currrent', 'current'),
         ('hidraulogy', 'hydrology'),
         ('californa', 'California'),
         ('electrrical', 'electrical'),
         ('navigationally', 'navigation'),
         ('whwhat', 'what'),
         ('bcos', 'because'),
         ('vaccancies', 'vacancies'),
         ('articels', 'articles'),
         ('boilng', 'boiling'),
         ('hyperintensity', 'hyper intensity'),
         ('rascism', 'racism'),
         ('messenging', 'messaging'),
         ('cleaniness', 'cleanliness'),
         ('vetenary', 'veterinary'),
         ('investorswhat', 'investors what'),
         ('chrestianity', 'Christianity'),
         ('apporval', 'approval'),
         ('repaire', 'repair'),
         ('biggerchance', 'bigger chance'),
         ('manufacturering', 'manufacturing'),
         ('buildertrend', 'builder trend'),
         ('allocatively', 'allocative'),
         ('subliminals', 'subliminal'),
         ('mechnically', 'mechanically'),
         ('binaurial', 'binaural'),
         ('naaked', 'naked'),
         ('aantidepressant', 'antidepressant'),
         ('geunine', 'genuine'),
         ('quantitaive', 'quantitative'),
         ('paticipated', 'participated'),
         ('repliedjesus', 'replied Jesus'),
         ('baised', 'biased'),
         ('worldreport', 'world report'),
         ('eecutives', 'executives'),
         ('paitents', 'patients'),
         ('telgu', 'Telugu'),
         ('nomeniculature', 'nomenclature'),
         ('crimimaly', 'criminally'),
         ('resourse', 'resource'),
         ('procurenent', 'procurement'),
         ('improvemet', 'improvement'),
         ('metamers', 'metamer'),
         ('tautomers', 'tautomer'),
         ('knowwhen', 'know when'),
         ('whatdoes', 'what does'),
         ('pletelets', 'platelets'),
         ('pssesive', 'possessive'),
         ('oxigen', 'oxygen'),
         ('ethniticy', 'ethnicity'),
         ('situatiation', 'situation'),
         ('ecoplanet', 'eco planet'),
         ('situatio', 'situation'),
         ('dateing', 'dating'),
         ('hostress', 'hostess'),
         ('initialisation', 'initialization'),
         ('hydrabd', 'Hyderabad'),
         ('deppresed', 'depressed'),
         ('dwnloadng', 'downloading'),
         ('expirey', 'expiry'),
         ('engeenering', 'engineering'),
         ('hyderebad', 'Hyderabad'),
         ('automatabl', 'automatable'),
         ('architetureocasions', 'architectureoccasions'),
         ('restaraunts', 'restaurants'),
         ('recommedations', 'recommendations'),
         ('intergrity', 'integrity'),
         ('reletively', 'relatively'),
         ('priceworthy', 'price worthy'),
         ('princples', 'principles'),
         ('reconigze', 'recognize'),
         ('paticular', 'particular'),
         ('musictheory', 'music theory'),
         ('requied', 'required'),
         ('netural', 'natural'),
         ('fluoresent', 'fluorescent'),
         ('girlfiend', 'girlfriend'),
         ('develpment', 'development'),
         ('eridicate', 'eradicate'),
         ('techologys', 'technologies'),
         ('hybridyzation', 'hybridization'),
         ('ideaa', 'ideas'),
         ('tchnology', 'technology'),
         ('appropiate', 'appropriate'),
         ('respone', 'response'),
         ('celebreties', 'celebrities'),
         ('exterion', 'exterior'),
         ('uservoice', 'user voice'),
         ('effeciently', 'efficiently'),
         ('torquise', 'turquoise '),
         ('governmentand', 'government and'),
         ('eletricity', 'electricity'),
         ('coulums', 'columns'),
         ('nolonger', 'no longer'),
         ('wheras', 'whereas'),
         ('infnite', 'infinite'),
         ('decolourised', 'no color'),
         ('onepiece', 'one piece'),
         ('assignements', 'assignments'),
         ('celebarted', 'celebrated'),
         ('pharmacistical', 'pharmaceutical'),
         ('jainsingle', 'Jain single'),
         ('asssistance', 'assistance'),
         ('glases', 'glasses'),
         ('polymorpism', 'polymorphism'),
         ('amerians', 'Americans'),
         ('masquitos', 'mosquitoes'),
         ('interseted', 'interested'),
         ('thehighest', 'the highest'),
         ('etnicity', 'ethnicity'),
         ('anopportunity', 'anopportunity'),
         ('multidiscipline', 'multi discipline'),
         ('smartchange', 'smart change'),
         ('collegefest', 'college fest'),
         ('disdvantages', 'disadvantages'),
         ('successfcators', 'success factors'),
         ('sustitute', 'substitute'),
         ('caoching', 'coaching'),
         ('bullyed', 'bullied'),
         ('comunicate', 'communicate'),
         ('prisioner', 'prisoner'),
         ('tamilnaadu', 'Tamil Nadu'),
         ('methodologyies', 'methodologies'),
         ('tranfers', 'transfers'),
         ('truenorth', 'true north'),
         ('backdonation', 'back donation'),
         ('oreals', 'ordeals'),
         ('browsec', 'browser'),
         ('solarwinds', 'solar winds'),
         ('susten', 'sustain'),
         ('carnegi', 'Carnegie'),
         ('doesent', "doesn't"),
         ('automtotive', 'automotive'),
         ('nimuselide', 'nimesulide'),
         ('subsciption', 'subscription'),
         ('quatrone', 'Quattrone'),
         ('qatalyst', 'catalyst'),
         ('vardamana', 'Vardaman'),
         ('suplements', 'supplements'),
         ('repore', 'report'),
         ('pikettys', 'Piketty'),
         ('paramilltary', 'paramilitary'),
         ('aboutlastnight', 'about last night'),
         ('vidyapeth', 'Vidyapeeth'),
         ('extraterrestial', 'extraterrestrial'),
         ('powerloom', 'power loom'),
         ('zonbie', 'zombie'),
         ('cococola', 'Coca Cola'),
         ('hameorrhage', 'hemorrhage'),
         ('abhayanand', 'Abhay Anand'),
         ('romedynow', 'remedy now'),
         ('couster', 'counter'),
         ('encouaged', 'encouraged'),
         ('toprepare', 'to prepare'),
         ('eveteasing', 'eve teasing'),
         ('roulete', 'roulette'),
         ('sorkar', 'Sarkar'),
         ('waveboard', 'wave board'),
         ('acclerate', 'accelerate'),
         ('togrow', 'to grow'),
         ('felatio', 'fellatio'),
         ('baherain', 'Bahrain'),
         ('teatment', 'treatment'),
         ('iwitness', 'eye witness'),
         ('autoplaying', 'autoplay'),
         ('twise', 'twice'),
         ('timeskip', 'time skip'),
         ('disphosphorus', 'diphosphorus'),
         ('implemnt', 'implement'),
         ('proview', 'preview'),
         ('pinshoppr', 'pin shoppe'),
         ('protestng', 'protesting'),
         ('chromatographymass', 'chromatography mass'),
         ('ncache', 'cache'),
         ('dowloands', 'downloads'),
         ('biospecifics', 'bio specifics'),
         ('conforim', 'conform'),
         ('dreft', 'draft'),
         ('sinhaleseand', 'Sinhalese'),
         ('swivl', 'swivel'),
         ('officerjms', 'officers'),
         ('refrigrant', 'refrigerant'),
         ('kendras', 'Kendra'),
         ('alchoholism', 'alcoholism'),
         ('dollor', 'dollar'),
         ('jeyalalitha', 'Jayalalitha'),
         ('bettner', 'better'),
         ('itemstream', 'timestream'),
         ('notetaking', 'note taking'),
         ('cringworthy', 'cringeworthy'),
         ('easyday', 'easy day'),
         ('scenessex', 'scenes sex'),
         ('vivavideo', 'via video'),
         ('washboth', 'wash both'),
         ('textout', 'text out'),
         ('createwindow', 'create window'),
         ('calsium', 'calcium'),
         ('biofibre', 'bio fibre'),
         ('emailbesides', 'email besides'),
         ('kathhi', 'Kathi'),
         ('cenre', 'center'),
         ('polyarmory', 'polyamory'),
         ('superforecasters', 'super forecasters'),
         ('blogers', 'bloggers'),
         ('medicalwhich', 'medical which'),
         ('iiving', 'living'),
         ('pronouciation', 'pronunciation'),
         ('youor', 'you or'),
         ('thuderbird', 'Thunderbird'),
         ('oneside', 'one side'),
         ('spearow', 'Spearow'),
         ('aanythign', 'anything'),
         ('inmaking', 'in making'),
         ('datamining', 'data mining'),
         ('greybus', 'grey bus'),
         ('onmeter', 'on meter'),
         ('biling', 'billing'),
         ('fidlago', 'Fidalgo'),
         ('edfice', 'edifice'),
         ('microsolutions', 'micro solutions'),
         ('easly', 'easily'),
         ('eukarotic', 'eukaryotic'),
         ('accedental', 'accidental'),
         ('intercasts', 'interests'),
         ('oppresive', 'oppressive'),
         ('generalizably', 'generalizable'),
         ('tacometer', 'tachometer'),
         ('loking', 'looking'),
         ('scrypt', 'script'),
         ('usafter', 'us after'),
         ('everyweek', 'every week'),
         ('hopesthe', 'hopes the'),
         ('openflow', 'OpenFlow'),
         ('checkride', 'check ride'),
         ('springdrive', 'spring drive'),
         ('emobile', 'mobile'),
         ('dermotology', 'dermatology'),
         ('somatrophin', 'somatropin'),
         ('saywe', 'say we'),
         ('multistores', 'multistory'),
         ('bolognaise', 'Bolognese'),
         ('hardisk', 'harddisk'),
         ('penisula', 'peninsula'),
         ('refferring', 'referring'),
         ('freshere', 'fresher'),
         ('pokemkon', 'Pokemon'),
         ('nuero', 'neuro'),
         ('whosampled', 'who sampled'),
         ('researchkit', 'research kit'),
         ('speach', 'speech'),
         ('acept', 'accept'),
         ('indiashoppe', 'Indian shoppe'),
         ('todescribe', 'to describe'),
         ('hollywod', 'Hollywood'),
         ('whastup', 'whassup'),
         ('kjedahls', 'Kjeldahl'),
         ('lancher', 'launcher'),
         ('stalkees', 'stalkers'),
         ('baclinks', 'backlinks'),
         ('instutional', 'institutional'),
         ('wassap', 'Wassup'),
         ('methylethyl', 'methyl ethyl'),
         ('fundbox', 'fund box'),
         ('keypoints', 'key points'),
         ('particually', 'particularly'),
         ('loseit', 'lose it'),
         ('gowipe', 'go wipe'),
         ('autority', 'authority'),
         ('prinicple', 'principle'),
         ('complaince', 'compliance'),
         ('itnormal', 'it normal'),
         ('forpeople', 'for people'),
         ('chaces', 'chances'),
         ('yearhow', 'year how'),
         ('fastcomet', 'fast comet'),
         ('withadd', 'with add'),
         ('omnicient', 'omniscient'),
         ('tofeel', 'to feel'),
         ('becauseof', 'because of'),
         ('laungauage', 'language'),
         ('combodia', 'Cambodia'),
         ('bhuvneshwer', 'Bhubaneshwar'),
         ('cognito', 'Cognito'),
         ('thaelsemia', 'thalassemia'),
         ('meritstore', 'merit store'),
         ('masterbuate', 'masturbate'),
         ('planethere', 'planet here'),
         ('mostof', 'most of'),
         ('shallowin', 'shallow in'),
         ('wordwhen', 'word when'),
         ('biodesalination', 'desalination'),
         ('tendulkars', 'Tendulkar'),
         ('kerja', 'Kerja'),
         ('sertifikat', 'certificate'),
         ('indegenous', 'indigenous'),
         ('lowpage', 'low page'),
         ('asend', 'ascend'),
         ('leadreship', 'leadership'),
         ('openlab', 'open lab'),
         ('foldinghome', 'folding home'),
         ('sachins', 'Sachin'),
         ('pleatue', 'plateau'),
         ('passwor', 'password'),
         ('manisfestation', 'manifestation'),
         ('valryian', 'valerian'),
         ('chemotaxic', 'chemotaxis'),
         ('condesending', 'condescending'),
         ('spiltzvilla', 'splitsville'),
         ('mammaliaforme', 'mammaliaform'),
         ('instituteagra', 'institute agra'),
         ('learningand', 'learning and'),
         ('ramamurthynagar', 'Ramamurthy Nagar'),
         ('glucoses', 'glucose'),
         ('imitaion', 'imitation'),
         ('awited', 'awaited'),
         ('realvision', 'real vision'),
         ('simslot', 'sim slot'),
         ('yourr', 'your'),
         ('pacjage', 'package'),
         ('branchth', 'branch'),
         ('magzin', 'magazine'),
         ('frozon', 'frozen'),
         ('codescomputational', 'code computational'),
         ('tempratures', 'temperatures'),
         ('neurophaphy', 'neuropathy'),
         ('freezone', 'free zone'),
         ('speices', 'species'),
         ('compaitable', 'compatible'),
         ('sensilization', 'sensitization'),
         ('tuboscope', 'tube scope'),
         ('gamechangers', 'game changer'),
         ('windsheild', 'windshield'),
         ('explorerie', 'explorer'),
         ('cuccina', 'Cucina'),
         ('earthstone', 'hearthstone'),
         ('vocabs', 'vocab'),
         ('previouse', 'previous'),
         ('oneview', 'one view'),
         ('relance', 'reliance'),
         ('waterstop', 'water stop'),
         ('imput', 'input'),
         ('survivers', 'survivors'),
         ('benedryl', 'Benadryl'),
         ('requestparam', 'request param'),
         ('typeadd', 'type add'),
         ('autists', 'artists'),
         ('forany', 'for any'),
         ('inteview', 'interview'),
         ('aphantasia', 'Phantasia'),
         ('lisanna', 'Lisanne'),
         ('civilengineering', 'civil engineering'),
         ('austrailia', 'Australia'),
         ('alchoholic', 'alcoholic'),
         ('adaptersuch', 'adapter such'),
         ('sphilosopher', 'philosopher'),
         ('calenderisation', 'calendarization'),
         ('smooking', 'smoking'),
         ('pemdulum', 'pendulum'),
         ('analsyis', 'analysis'),
         ('psycholology', 'psychology'),
         ('ubantu', 'ubuntu'),
         ('emals', 'emails'),
         ('questionth', 'questions'),
         ('jawarlal', 'Jawaharlal'),
         ('svaldbard', 'Svalbard'),
         ('prabhudeva', 'Prabhudeva'),
         ('robtics', 'robotics'),
         ('umblock', 'unblock'),
         ('professionaly', 'professionally'),
         ('biovault', 'bio vault'),
         ('bibal', 'bible'),
         ('higherstudies', 'higher studies'),
         ('lestoil', 'less oil'),
         ('biteshow', 'bike show'),
         ('humanslike', 'humans like'),
         ('purpse', 'purpose'),
         ('barazilian', 'Brazilian'),
         ('gravitional', 'gravitational'),
         ('cylinderical', 'cylindrical'),
         ('peparing', 'preparing'),
         ('healthequity', 'health equity'),
         ('appcleaner', 'app cleaner'),
         ('instantq', 'instant'),
         ('abolisihed', 'abolished'),
         ('kwench', 'quench'),
         ('prisamatic', 'prismatic'),
         ('bhubneshwar', 'Bhubaneshwar'),
         ('liscense', 'license'),
         ('cyberbase', 'cyber base'),
         ('safezone', 'safe zone'),
         ('deactivat', 'deactivate'),
         ('salicyclic', 'salicylic'),
         ('cocacola', 'coca cola'),
         ('noice', 'noise'),
         ('examinaton', 'examination'),
         ('pharmavigilance', 'pharmacovigilance'),
         ('sixthsense', 'sixth sense'),
         ('musiclly', 'musically'),
         ('khardushan', 'Kardashian'),
         ('chandragupt', 'Chandragupta'),
         ('bayesians', 'bayesian'),
         ('engineeringbut', 'engineering but'),
         ('caretrust', 'care trust'),
         ('girlbut', 'girl but'),
         ('aviations', 'aviation'),
         ('joinee', 'joiner'),
         ('tutior', 'tutor'),
         ('tylenal', 'Tylenol'),
         ('neccesity', 'necessity'),
         ('kapsule', 'capsule'),
         ('prayes', 'prayers'),
         ('depositmobile', 'deposit mobile'),
         ('settopbox', 'set top box'),
         ('meotic', 'meiotic'),
         ('accidentially', 'accidentally'),
         ('offcloud', 'off cloud'),
         ('keshavam', 'Keshava'),
         ('domaincentral', 'domain central'),
         ('onetaste', 'one taste'),
         ('lumpsum', 'lump sum'),
         ('medschool', 'med school'),
         ('digicard', 'Digi card'),
         ('abroadus', 'abroad'),
         ('campusexcept', 'campus except'),
         ('aptittude', 'aptitude'),
         ('neutrions', 'neutrinos'),
         ('onepaper', 'one paper'),
         ('remidies', 'remedies'),
         ('convinient', 'convenient'),
         ('financaily', 'financially'),
         ('postives', 'positives'),
         ('nikefuel', 'Nike fuel'),
         ('ingrediants', 'ingredients'),
         ('aspireat', 'aspirate'),
         ('firstand', 'first'),
         ('mohammmad', 'Mohammad'),
         ('mutliple', 'multiple'),
         ('dimonatization', 'demonization'),
         ('cente', 'center'),
         ('marshmellow', 'marshmallow'),
         ('citreon', 'Citroen'),
         ('theirony', 'the irony'),
         ('slienced', 'silenced'),
         ('identifiy', 'identify'),
         ('energ', 'energy'),
         ('distribuiton', 'distribution'),
         ('devoloping', 'developing'),
         ('maharstra', 'Maharastra'),
         ('siesmologist', 'seismologist'),
         ('geckoos', 'geckos'),
         ('placememnt', 'placement'),
         ('introvercy', 'introvert'),
         ('nuerosurgeon', 'neurosurgeon'),
         ('realsense', 'real sense'),
         ('congac', 'cognac'),
         ('plaese', 'please'),
         ('addicition', 'addiction'),
         ('othet', 'other'),
         ('howwill', 'how will'),
         ('betablockers', 'beta blockers'),
         ('phython', 'Python'),
         ('concelling', 'counseling'),
         ('einstine', 'Einstein'),
         ('takinng', 'taking'),
         ('birtday', 'birthday'),
         ('prefessor', 'professor'),
         ('dreamscreen', 'dream screen'),
         ('satyabama', 'Satyabhama'),
         ('faminism', 'feminism'),
         ('noooooooooo', 'no'),
         ('certifaction', 'certification'),
         ('smalll', 'small'),
         ('sterlization', 'sterilization'),
         ('athelete', 'athlete'),
         ('comppany', 'company'),
         ('handlebreakup', 'handle a breakup'),
         ('wellrounded', 'well rounded'),
         ('breif', 'brief'),
         ('engginering', 'engineering'),
         ('genrally', 'generally'),
         ('forgote', 'forgot'),
         ('compuny', 'the company'),
         ('wholeseller', 'wholesaler'),
         ('conventioal', 'conventional'),
         ('healther', 'healthier'),
         ('realitic', 'realistic'),
         ('israil', 'Israel'),
         ('morghulis', 'Margulis'),
         ('begineer', 'beginner'),
         ('unwaiveringly', 'unwavering'),
         ('writen', 'written'),
         ('gastly', 'ghastly'),
         ('obscurial', 'obscure'),
         ('permanetly', 'permanently'),
         ('bday', 'birthday'),
         ('studing', 'studying'),
         ('blackcore', 'black core'),
         ('macbok', 'MacBook'),
         ('realted', 'related'),
         ('resoning', 'reasoning'),
         ('servicenow', 'service now'),
         ('medels', 'medals'),
         ('hairloss', 'hair loss'),
         ('messanger', 'messenger'),
         ('masterbate', 'masturbate'),
         ('oppurtunities', 'opportunities'),
         ('newzealand', 'new zealand'),
         ('offcampus', 'off campus'),
         ('lonliness', 'loneliness'),
         ('percentilers', 'percentiles'),
         ('caccount', 'account'),
         ('imrovement', 'improvement'),
         ('cashbacks', 'cashback'),
         ('inhand', 'in hand'),
         ('baahubali', 'bahubali'),
         ('diffrent', 'different'),
         ('strategywho', 'strategy who'),
         ('meetme', 'meet me'),
         ('wealthfront', 'wealth front'),
         ('masterbation', 'masturbation'),
         ('successfull', 'successful'),
         ('lenght', 'length'),
         ('increse', 'increase'),
         ('mastrubation', 'masturbation'),
         ('intresting', 'interesting'),
         ('quesitons', 'questions'),
         ('fullstack', 'full stack'),
         ('harambe', 'Harambee'),
         ('criterias', 'criteria'),
         ('rajyasabha', 'Rajya Sabha'),
         ('techmahindra', 'tech Mahindra'),
         ('messeges', 'messages'),
         ('intership', 'internship'),
         ('benifits', 'benefits'),
         ('dowload', 'download'),
         ('dellhi', 'Delhi'),
         ('traval', 'travel'),
         ('prepration', 'preparation'),
         ('engineeringwhat', 'engineering what'),
         ('habbit', 'habit'),
         ('diference', 'difference'),
         ('permantley', 'permanently'),
         ('doesnot', 'does not'),
         ('thebest', 'the best'),
         ('addmision', 'admission'),
         ('gramatically', 'grammatically'),
         ('dayswhich', 'days which'),
         ('intrest', 'interest'),
         ('seperatists', 'separatists'),
         ('plagarism', 'plagiarism'),
         ('demonitize', 'demonetize'),
         ('explaination', 'explanation'),
         ('numericals', 'numerical'),
         ('defination', 'definition'),
         ('inmortal', 'immortal'),
         ('elasticsearch', 'elastic search')
    ]



REGEX_REPLACER = [
    (re.compile(pat.replace("*", "\*"), flags=re.IGNORECASE), repl)
    for pat, repl in WORDS_REPLACER
]

REGEX_REPLACER_2 = [
    (re.compile(pat.replace("*", "\*"), flags=re.IGNORECASE), repl)
    for pat, repl in WORDS_REPLACER_2
]

REGEX_REPLACER_3 = [
    (re.compile(pat.replace("*", "\*"), flags=re.IGNORECASE), repl)
    for pat, repl in WORDS_REPLACER_3
]

REGEX_REPLACER_4 = [
    (re.compile(pat.replace("*", "\*"), flags=re.IGNORECASE), repl)
    for pat, repl in WORDS_REPLACER_4
]

"""
WORDS_REPLACER_5 = [('["code"]', '["CODE"]'),
                    ('["formula"]', '["FORMULA"]')
                   ]

REGEX_REPLACER_5 = [
    (re.compile(pat.replace("*", "\*"), flags=re.IGNORECASE), repl)
    for pat, repl in WORDS_REPLACER_5
]
"""

RE_SPACE = re.compile(r"\s")
RE_MULTI_SPACE = re.compile(r"\s+")

symbols_to_isolate = '.,?!-;*…:—()[]%#$&_/@＼・ω+=^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
symbols_to_delete = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'
symbols_to_delete2 = '"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’'


isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}
remove_dict1 = {ord(c):f'' for c in symbols_to_delete2}

NMS_TABLE = dict.fromkeys(
    i for i in range(sys.maxunicode + 1) if unicodedata.category(chr(i)) == "Mn"
)

HEBREW_TABLE = {i: "א" for i in range(0x0590, 0x05FF)}
ARABIC_TABLE = {i: "ا" for i in range(0x0600, 0x06FF)}
CHINESE_TABLE = {i: "是" for i in range(0x4E00, 0x9FFF)}
KANJI_TABLE = {i: "ッ" for i in range(0x2E80, 0x2FD5)}
HIRAGANA_TABLE = {i: "ッ" for i in range(0x3041, 0x3096)}
KATAKANA_TABLE = {i: "ッ" for i in range(0x30A0, 0x30FF)}

TABLE = dict()
TABLE.update(CUSTOM_TABLE)
TABLE.update(NMS_TABLE)
# Non-english languages
TABLE.update(CHINESE_TABLE)
TABLE.update(HEBREW_TABLE)
TABLE.update(ARABIC_TABLE)
TABLE.update(HIRAGANA_TABLE)
TABLE.update(KATAKANA_TABLE)
TABLE.update(KANJI_TABLE)



EMOJI_REGEXP = emoji.get_emoji_regexp()

UNICODE_EMOJI_MY = {
    k: f" EMJ {v.strip(':').replace('_', ' ')} "
    for k, v in emoji.UNICODE_EMOJI_ALIAS.items()
}

def my_demojize(string: str) -> str:
    def replace(match):
        return UNICODE_EMOJI_MY.get(match.group(0), match.group(0))
    
    return re.sub("\ufe0f", "", EMOJI_REGEXP.sub(replace, string))

def normalize(text: str) -> str:
    
    text = text.replace("[CODE]", " ACODEA ")
    text = text.replace("[FORMULA]", " AFORMULAA ")
    #text = text.replace("[]", " [URL] ")
    
    text = html.unescape(text)
    
    text = text.lower()
    
    text = my_demojize(text)
    
    # replacing urls with "url" string
    text = re.sub('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', '[WEB]' ,text)
    text = text.replace("[WEB]", " AWEBA ")
    
    text = RE_SPACE.sub(" ", text)
    text = unicodedata.normalize("NFKD", text)
    text = text.translate(TABLE)
    text = RE_MULTI_SPACE.sub(" ", text).strip()
    
    # remove some unimportent symbles
    text = text.translate(remove_dict)
    
    # remove some unimportent symbles
    text = text.translate(remove_dict1)
    
    text = text.translate(isolate_dict)
    

    # Replacing and mispell
    
    for pattern, repl in REGEX_REPLACER:
        text = pattern.sub(repl, text)
    
    for pattern, repl in REGEX_REPLACER_2:
        text = pattern.sub(repl, text)
            
    for pattern, repl in REGEX_REPLACER_3:
        text = pattern.sub(repl, text)
        
    # isolated_characters
    #text = text.translate(isolate_dict)
    
    for pattern, repl in REGEX_REPLACER_4:
        text = pattern.sub(repl, text)
    """
    for pattern, repl in REGEX_REPLACER_5:
        text = pattern.sub(repl, text)
    """
    
    
        
    text = RE_MULTI_SPACE.sub(" ", text).strip()
    
    text = text.replace(" acodea ", " [CODE] ")
    text = text.replace(" aformulaa ", " [FORMULA] ")
    text = text.replace(" AWEBA ", " [WEB] ")
    
    text = RE_MULTI_SPACE.sub(" ", text).strip()
    
    return text

In [ ]:
%%time

train_final["question_body_clean1"] = train_final.question_body_clean.apply(normalize)
train_final["answer_clean1"]  = train_final.answer_clean.apply(normalize)
train_final["question_title_clean1"] = train_final.question_title.apply(normalize)

test_final["question_body_clean1"] = test_final.question_body_clean.apply(normalize)
test_final["answer_clean1"]  = test_final.answer_clean.apply(normalize)
test_final["question_title_clean1"] = test_final.question_title.apply(normalize)

train_final["question_body_clean1"].fillna("please see figure below", inplace=True)
test_final["question_body_clean1"].fillna("please see figure below", inplace=True)

train_final["answer_clean1"].fillna("please see figure below", inplace=True)
test_final["answer_clean1"].fillna("please see figure below", inplace=True)

train_final["question_title_clean1"].fillna("please see figure below", inplace=True)
test_final["question_title_clean1"].fillna("please see figure below", inplace=True)

In [ ]:
%%time

def replace_specialtokens(text):
    text = text.replace(" [CODE] ", " code ")
    text = text.replace(" [FORMULA] ", " formula ")
    text = text.replace(" [WEB] ", " web ")
    return text

train_final["question_body_clean2"] = train_final.question_body_clean1.apply(replace_specialtokens)
train_final["answer_clean2"]  = train_final.answer_clean1.apply(replace_specialtokens)
train_final["question_title_clean2"] = train_final.question_title_clean1.apply(replace_specialtokens)

test_final["question_body_clean2"] = test_final.question_body_clean1.apply(replace_specialtokens)
test_final["answer_clean2"]  = test_final.answer_clean1.apply(replace_specialtokens)
test_final["question_title_clean2"] = test_final.question_title_clean1.apply(replace_specialtokens)

In [ ]:
train_final["question_body_all_clean"] = train_final.question_body_all.apply(normalize)
train_final["answer_all_clean"]  = train_final.answer_all.apply(normalize)

test_final["question_body_all_clean"] = test_final.question_body_all.apply(normalize)
test_final["answer_all_clean"]  = test_final.answer_all.apply(normalize)

## META DATA

In [ ]:
from sklearn.decomposition import PCA
from scipy.sparse import vstack
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def text_metadata(train, test):
    
    a_AboutMe_text = train["a_AboutMe"].apply(normalize)
    q_AboutMe_text = train["q_AboutMe"].apply(normalize)
    all_text = pd.concat([a_AboutMe_text, q_AboutMe_text])

    word_vectorizer = TfidfVectorizer(
                sublinear_tf=True,
                strip_accents='unicode',
                analyzer='word',
                token_pattern=r'\w{1,}',
                stop_words='english',
                ngram_range=(1,1))
    
    word_vectorizer.fit(all_text)
    
    q_AboutMe_cols =  [f"q_AboutMe_PCA_{i}" for i in range(1,101)] 
    a_AboutMe_cols =  [f"a_AboutMe_PCA_{i}" for i in range(1,101)]
    
    q_AboutMe_text = word_vectorizer.transform(q_AboutMe_text)
    a_AboutMe_text = word_vectorizer.transform(a_AboutMe_text)
    
    tq_AboutMe_text = word_vectorizer.transform(test["q_AboutMe"].apply(normalize))
    ta_AboutMe_text = word_vectorizer.transform(test["a_AboutMe"].apply(normalize))
    
    new = vstack([q_AboutMe_text, a_AboutMe_text])
    
    pca = PCA(n_components=100)
    pca.fit(new.toarray())
    
    q_AboutMe_text = pca.transform(q_AboutMe_text.toarray())
    a_AboutMe_text = pca.transform(a_AboutMe_text.toarray())
    
    tq_AboutMe_text = pca.transform(tq_AboutMe_text.toarray())
    ta_AboutMe_text = pca.transform(ta_AboutMe_text.toarray())
    
    
    train[q_AboutMe_cols] = pd.DataFrame(q_AboutMe_text, columns=q_AboutMe_cols, index=train.index)
    test[q_AboutMe_cols] = pd.DataFrame(tq_AboutMe_text, columns=q_AboutMe_cols, index=test.index)
    
    train[a_AboutMe_cols] = pd.DataFrame(a_AboutMe_text, columns=a_AboutMe_cols, index=train.index)
    test[a_AboutMe_cols] = pd.DataFrame(ta_AboutMe_text, columns=a_AboutMe_cols, index=test.index)
    
    
    # Tages
    
    tags_all_text = train["Tags"]

    word_vectorizer = CountVectorizer()
    
    word_vectorizer.fit(tags_all_text)
    
    tags_cols =  ["Tags_" + sub for sub in word_vectorizer.get_feature_names()] 
    
    train[tags_cols] = pd.DataFrame(word_vectorizer.transform(train["Tags"]).toarray(), columns=tags_cols, index=train.index)
    test[tags_cols] = pd.DataFrame(word_vectorizer.transform(test["Tags"]).toarray(), columns=tags_cols, index=test.index)
    
    
    
    return train, test, q_AboutMe_cols, a_AboutMe_cols, tags_cols
    

In [ ]:
%%time

train_final, test_final, q_AboutMe_cols, a_AboutMe_cols, tags_cols = text_metadata(train_final, test_final)

In [ ]:
train_final.shape, test_final.shape

In [ ]:
def linear_based_models(train, test, cat_cols, num_cols):
    
    # Column std MinMax Scalling
    std = MinMaxScaler()
    train[num_cols] = std.fit_transform(train[num_cols])
    test[num_cols] = std.transform(test[num_cols])
    
    # One Hot Encoder
    train = pd.get_dummies(train, columns=cat_cols, prefix=cat_cols)
    test = pd.get_dummies(test, columns=cat_cols, prefix=cat_cols)
    
    rem = list(set(train.columns).intersection(set(test.columns)))
    
    train = train[rem]
    test = test[rem]
    

    return train, test

num_cols = ['AnswerCount', 'CommentCount', 'FavoriteCount','PostTypeId', 'Score','ViewCount','a_DownVotes','a_UpVotes', 'a_Views','q_DownVotes', 'q_UpVotes', 'q_Views']
cat_cols = ['category', 'category_type']
train_final, test_final = linear_based_models(train_final, test_final, cat_cols, num_cols)

In [ ]:
train_final.shape, test_final.shape

In [ ]:
train_final = pd.merge(train_final, train_final_targets, left_on="qa_id", right_on="qa_id", how="left")

In [ ]:
train_final.shape, test_final.shape

# Model

In [ ]:
train_df = train_final
test_df = test_final

target_columns = train_df.columns.values.tolist()[-30:]

In [ ]:
category_features = [col for col in train_df.columns if col.startswith('category_')]
tags_cols = [col for col in train_df.columns if col.startswith('Tags_')]
a_AboutMe_cols = [col for col in train_df.columns if col.startswith('a_AboutMe_')]
q_AboutMe_cols = [col for col in train_df.columns if col.startswith('q_AboutMe_')]
num_cols = ['AnswerCount', 'CommentCount', 'FavoriteCount','PostTypeId', 'Score','ViewCount','a_DownVotes','a_UpVotes', 'a_Views','q_DownVotes', 'q_UpVotes', 'q_Views','AnswerCount_nan', 'CommentCount_nan', 'FavoriteCount_nan', 'Score_nan','ViewCount_nan','a_DownVotes_nan','a_UpVotes_nan', 'a_Views_nan','q_DownVotes_nan', 'q_UpVotes_nan', 'q_Views_nan']

In [ ]:
a_AboutMe_cols = ['a_AboutMe_PCA_64',
 'a_AboutMe_PCA_19',
 'a_AboutMe_PCA_40',
 'a_AboutMe_PCA_6',
 'a_AboutMe_PCA_77',
 'a_AboutMe_PCA_35',
 'a_AboutMe_PCA_100',
 'a_AboutMe_PCA_57',
 'a_AboutMe_PCA_76',
 'a_AboutMe_PCA_16',
 'a_AboutMe_PCA_67',
 'a_AboutMe_PCA_36',
 'a_AboutMe_PCA_25',
 'a_AboutMe_PCA_26',
 'a_AboutMe_PCA_7',
 'a_AboutMe_PCA_50',
 'a_AboutMe_PCA_32',
 'a_AboutMe_PCA_60',
 'a_AboutMe_PCA_54',
 'a_AboutMe_PCA_84',
 'a_AboutMe_PCA_66',
 'a_AboutMe_PCA_88',
 'a_AboutMe_PCA_61',
 'a_AboutMe_PCA_23',
 'a_AboutMe_PCA_37',
 'a_AboutMe_PCA_1',
 'a_AboutMe_PCA_21',
 'a_AboutMe_PCA_20',
 'a_AboutMe_PCA_55',
 'a_AboutMe_PCA_86',
 'a_AboutMe_PCA_2',
 'a_AboutMe_PCA_3',
 'a_AboutMe_PCA_99',
 'a_AboutMe_PCA_18',
 'a_AboutMe_PCA_78',
 'a_AboutMe_PCA_51',
 'a_AboutMe_PCA_53',
 'a_AboutMe_PCA_96',
 'a_AboutMe_PCA_15',
 'a_AboutMe_PCA_11',
 'a_AboutMe_PCA_89',
 'a_AboutMe_PCA_82',
 'a_AboutMe_PCA_13',
 'a_AboutMe_PCA_44',
 'a_AboutMe_PCA_28',
 'a_AboutMe_PCA_41',
 'a_AboutMe_PCA_68',
 'a_AboutMe_PCA_42',
 'a_AboutMe_PCA_27',
 'a_AboutMe_PCA_73',
 'a_AboutMe_PCA_95',
 'a_AboutMe_PCA_85',
 'a_AboutMe_PCA_49',
 'a_AboutMe_PCA_33',
 'a_AboutMe_PCA_48',
 'a_AboutMe_PCA_59',
 'a_AboutMe_PCA_46',
 'a_AboutMe_PCA_65',
 'a_AboutMe_PCA_75',
 'a_AboutMe_PCA_63',
 'a_AboutMe_PCA_4',
 'a_AboutMe_PCA_52',
 'a_AboutMe_PCA_5',
 'a_AboutMe_PCA_17',
 'a_AboutMe_PCA_92',
 'a_AboutMe_PCA_47',
 'a_AboutMe_PCA_80',
 'a_AboutMe_PCA_14',
 'a_AboutMe_PCA_98',
 'a_AboutMe_PCA_34',
 'a_AboutMe_PCA_83',
 'a_AboutMe_PCA_58',
 'a_AboutMe_PCA_94',
 'a_AboutMe_PCA_69',
 'a_AboutMe_PCA_45',
 'a_AboutMe_PCA_31',
 'a_AboutMe_PCA_91',
 'a_AboutMe_PCA_12',
 'a_AboutMe_PCA_70',
 'a_AboutMe_PCA_8',
 'a_AboutMe_PCA_39',
 'a_AboutMe_PCA_74',
 'a_AboutMe_PCA_43',
 'a_AboutMe_PCA_62',
 'a_AboutMe_PCA_10',
 'a_AboutMe_PCA_9',
 'a_AboutMe_PCA_22',
 'a_AboutMe_PCA_30',
 'a_AboutMe_PCA_24',
 'a_AboutMe_PCA_87',
 'a_AboutMe_PCA_79',
 'a_AboutMe_PCA_81',
 'a_AboutMe_PCA_90',
 'a_AboutMe_PCA_93',
 'a_AboutMe_PCA_38',
 'a_AboutMe_PCA_72',
 'a_AboutMe_PCA_29',
 'a_AboutMe_PCA_56',
 'a_AboutMe_PCA_97',
 'a_AboutMe_PCA_71']

q_AboutMe_cols = ['q_AboutMe_PCA_93',
 'q_AboutMe_PCA_50',
 'q_AboutMe_PCA_70',
 'q_AboutMe_PCA_65',
 'q_AboutMe_PCA_85',
 'q_AboutMe_PCA_71',
 'q_AboutMe_PCA_18',
 'q_AboutMe_PCA_69',
 'q_AboutMe_PCA_51',
 'q_AboutMe_PCA_79',
 'q_AboutMe_PCA_31',
 'q_AboutMe_PCA_99',
 'q_AboutMe_PCA_40',
 'q_AboutMe_PCA_92',
 'q_AboutMe_PCA_86',
 'q_AboutMe_PCA_34',
 'q_AboutMe_PCA_2',
 'q_AboutMe_PCA_64',
 'q_AboutMe_PCA_1',
 'q_AboutMe_PCA_72',
 'q_AboutMe_PCA_32',
 'q_AboutMe_PCA_29',
 'q_AboutMe_PCA_5',
 'q_AboutMe_PCA_7',
 'q_AboutMe_PCA_67',
 'q_AboutMe_PCA_96',
 'q_AboutMe_PCA_82',
 'q_AboutMe_PCA_35',
 'q_AboutMe_PCA_55',
 'q_AboutMe_PCA_39',
 'q_AboutMe_PCA_27',
 'q_AboutMe_PCA_4',
 'q_AboutMe_PCA_66',
 'q_AboutMe_PCA_57',
 'q_AboutMe_PCA_38',
 'q_AboutMe_PCA_12',
 'q_AboutMe_PCA_76',
 'q_AboutMe_PCA_20',
 'q_AboutMe_PCA_89',
 'q_AboutMe_PCA_28',
 'q_AboutMe_PCA_30',
 'q_AboutMe_PCA_98',
 'q_AboutMe_PCA_100',
 'q_AboutMe_PCA_61',
 'q_AboutMe_PCA_3',
 'q_AboutMe_PCA_37',
 'q_AboutMe_PCA_81',
 'q_AboutMe_PCA_97',
 'q_AboutMe_PCA_49',
 'q_AboutMe_PCA_91',
 'q_AboutMe_PCA_43',
 'q_AboutMe_PCA_90',
 'q_AboutMe_PCA_94',
 'q_AboutMe_PCA_58',
 'q_AboutMe_PCA_36',
 'q_AboutMe_PCA_8',
 'q_AboutMe_PCA_46',
 'q_AboutMe_PCA_25',
 'q_AboutMe_PCA_13',
 'q_AboutMe_PCA_10',
 'q_AboutMe_PCA_87',
 'q_AboutMe_PCA_21',
 'q_AboutMe_PCA_62',
 'q_AboutMe_PCA_11',
 'q_AboutMe_PCA_42',
 'q_AboutMe_PCA_33',
 'q_AboutMe_PCA_74',
 'q_AboutMe_PCA_26',
 'q_AboutMe_PCA_6',
 'q_AboutMe_PCA_68',
 'q_AboutMe_PCA_54',
 'q_AboutMe_PCA_73',
 'q_AboutMe_PCA_17',
 'q_AboutMe_PCA_44',
 'q_AboutMe_PCA_52',
 'q_AboutMe_PCA_47',
 'q_AboutMe_PCA_56',
 'q_AboutMe_PCA_15',
 'q_AboutMe_PCA_59',
 'q_AboutMe_PCA_88',
 'q_AboutMe_PCA_22',
 'q_AboutMe_PCA_77',
 'q_AboutMe_PCA_84',
 'q_AboutMe_PCA_75',
 'q_AboutMe_PCA_63',
 'q_AboutMe_PCA_60',
 'q_AboutMe_PCA_41',
 'q_AboutMe_PCA_53',
 'q_AboutMe_PCA_45',
 'q_AboutMe_PCA_14',
 'q_AboutMe_PCA_16',
 'q_AboutMe_PCA_24',
 'q_AboutMe_PCA_19',
 'q_AboutMe_PCA_83',
 'q_AboutMe_PCA_9',
 'q_AboutMe_PCA_48',
 'q_AboutMe_PCA_23',
 'q_AboutMe_PCA_78',
 'q_AboutMe_PCA_95',
 'q_AboutMe_PCA_80']

In [ ]:
meta_cols = [*category_features, *num_cols]

In [ ]:
len(meta_cols)

In [ ]:
def _convert_to_transformer_inputs(title, question, answer, tokenizer, max_sequence_lenght):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)"""

    def return_id(str1, str2, lenght):

        inputs = tokenizer.encode_plus(
            str1, str2,
            add_special_tokens = True,
            max_length = lenght,
            pad_to_max_length = True,
            return_token_type_ids = True,
            return_attention_mask = True,
            truncation_strategy = "longest_first"
        )

        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        return [input_ids, attention_mask, token_type_ids]


    input_ids_q , attention_mask_q, token_type_ids_q = return_id(title, question, max_sequence_lenght)
    input_ids_a , attention_mask_a, token_type_ids_a = return_id(answer,None, max_sequence_lenght)

    return [
            input_ids_q , attention_mask_q, token_type_ids_q,
            input_ids_a , attention_mask_a, token_type_ids_a
    ]

In [ ]:
def compute_input_arrays(df, columns, meta_cols, q_AboutMe_cols, a_AboutMe_cols, tokenizer, max_sequence_length):
    input_ids_q , attention_mask_q, token_type_ids_q = [], [], []
    input_ids_a , attention_mask_a, token_type_ids_a = [], [], []
    meta_features = []
    q_AboutMe_features = []
    a_AboutMe_features = []

    total_cols = [*columns, *meta_cols, *q_AboutMe_cols, *a_AboutMe_cols]

    #i = 0

    for _, instance in tqdm(df[total_cols].iterrows()):

    
        t, q, a, qc, ac = instance.question_title_clean2, instance.question_body_clean2, instance.answer_clean2,instance.question_body_all_clean, instance.answer_all_clean
        t = str(t) 
        q = str(q) + " [SEP] "+ str(qc)
        a = str(a) + " [SEP] "+ str(ac)

        ids_q, masks_q, segments_q, ids_a, masks_a, segments_a = _convert_to_transformer_inputs(t, q, a, tokenizer, max_sequence_length)

        input_ids_q.append(ids_q)
        attention_mask_q.append(masks_q)
        token_type_ids_q.append(segments_q)

        input_ids_a.append(ids_a)
        attention_mask_a.append(masks_a)
        token_type_ids_a.append(segments_a)

        meta_data = instance[meta_cols].values.tolist()
        q_AboutMe_data = instance[q_AboutMe_cols].values.tolist()
        a_AboutMe_data = instance[a_AboutMe_cols].values.tolist()

        meta_features.append(meta_data)
        q_AboutMe_features.append(q_AboutMe_data)
        a_AboutMe_features.append(a_AboutMe_data)
        #i = i+1
        #if i == 100:
        #    break

    return [np.asarray(input_ids_q, dtype=np.int32), 
            np.asarray(attention_mask_q, dtype=np.int32), 
            np.asarray(token_type_ids_q, dtype=np.int32),
            np.asarray(input_ids_a, dtype=np.int32), 
            np.asarray(attention_mask_a, dtype=np.int32), 
            np.asarray(token_type_ids_a, dtype=np.int32),
            np.asarray(meta_features, dtype=np.float32),
            np.asarray(q_AboutMe_features, dtype=np.float32),
            np.asarray(a_AboutMe_features, dtype=np.float32)
            ]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [ ]:
def create_model_soft(BERT_PATH):
    q_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)

    meta_features_layer = tf.keras.layers.Input((len(meta_cols),), dtype=tf.float32)
    q_AboutMe_features_layer = tf.keras.layers.Input((len(q_AboutMe_cols),), dtype=tf.float32)
    a_AboutMe_features_layer = tf.keras.layers.Input((len(a_AboutMe_cols),), dtype=tf.float32)
    
    config = BertConfig() # print(config) to see settings
    config.output_hidden_states = False # Set to True to obtain hidden states
    # caution: when using e.g. XLNet, XLNetConfig() will automatically use xlnet-large config
    
    # normally ".from_pretrained('bert-base-uncased')", but because of no internet, the 
    # pretrained model has been downloaded manually and uploaded to kaggle. 
    bert_model = TFBertModel.from_pretrained(BERT_PATH+'bert-base-uncased-tf_model.h5',config=config )
    
    # if config.output_hidden_states = True, obtain hidden states via bert_model(...)[-1]
    q_embedding = bert_model(q_id, attention_mask=q_mask, token_type_ids=q_atn)[0]
    a_embedding = bert_model(a_id, attention_mask=a_mask, token_type_ids=a_atn)[0]
    
    q = tf.keras.layers.GlobalAveragePooling1D()(q_embedding)
    a = tf.keras.layers.GlobalAveragePooling1D()(a_embedding)
    
    x = tf.keras.layers.Concatenate()([q, q_AboutMe_features_layer, a, a_AboutMe_features_layer, meta_features_layer, ])
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    x = tf.keras.layers.Dense(21, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=[q_id, q_mask, q_atn, a_id, a_mask, a_atn, meta_features_layer, q_AboutMe_features_layer, a_AboutMe_features_layer], outputs=x)
    
    return model

In [ ]:
def create_model_hard(BERT_PATH):
    q_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)

    meta_features_layer = tf.keras.layers.Input((len(meta_cols),), dtype=tf.float32)
    q_AboutMe_features_layer = tf.keras.layers.Input((len(q_AboutMe_cols),), dtype=tf.float32)
    a_AboutMe_features_layer = tf.keras.layers.Input((len(a_AboutMe_cols),), dtype=tf.float32)
    
    config = BertConfig() # print(config) to see settings
    config.output_hidden_states = False # Set to True to obtain hidden states
    # caution: when using e.g. XLNet, XLNetConfig() will automatically use xlnet-large config
    
    # normally ".from_pretrained('bert-base-uncased')", but because of no internet, the 
    # pretrained model has been downloaded manually and uploaded to kaggle. 
    bert_model = TFBertModel.from_pretrained(BERT_PATH+'bert-base-uncased-tf_model.h5',config=config )
    
    # if config.output_hidden_states = True, obtain hidden states via bert_model(...)[-1]
    q_embedding = bert_model(q_id, attention_mask=q_mask, token_type_ids=q_atn)[0]
    a_embedding = bert_model(a_id, attention_mask=a_mask, token_type_ids=a_atn)[0]
    
    q = tf.keras.layers.GlobalAveragePooling1D()(q_embedding)
    a = tf.keras.layers.GlobalAveragePooling1D()(a_embedding)
    
    x = tf.keras.layers.Concatenate()([q, q_AboutMe_features_layer, a, a_AboutMe_features_layer, meta_features_layer, ])
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    x = tf.keras.layers.Dense(8, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=[q_id, q_mask, q_atn, a_id, a_mask, a_atn, meta_features_layer, q_AboutMe_features_layer, a_AboutMe_features_layer], outputs=x)
    
    return model

In [ ]:
input_categories = ["question_title_clean2", "question_body_clean2", "answer_clean2", "question_body_all_clean", "answer_all_clean"]

MAX_SEQUENCE_LENGTH = 512

In [ ]:
BERT_PATH = '../input/bert-base-uncased-huggingface-transformer/'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(BERT_PATH+'bert-base-uncased-vocab.txt')

In [ ]:
#outputs = compute_output_arrays(train_df, target_columns)
#inputs = compute_input_arrays(train_df, input_categories,meta_cols, q_AboutMe_cols, a_AboutMe_cols ,tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_arrays(test_df, input_categories,meta_cols,q_AboutMe_cols, a_AboutMe_cols , tokenizer, MAX_SEQUENCE_LENGTH)

In [ ]:
model_weights_path = '../input/quest-bert-soft-hard-models/'

In [ ]:
soft_target_columns = [
 #'question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 #'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 #'question_type_compare',
 #'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 #'question_type_spelling',
 'question_well_written',
 #'answer_helpful',
 'answer_level_of_information',
 #'answer_plausible',
 #'answer_relevance',
 #'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [ ]:
soft_test_predictions = []

for i in range(5):
    j = 1
    model_path = f'{model_weights_path}bert-{i}-{j}.h5'
    model1 = create_model_soft(BERT_PATH)
    model1.load_weights(model_path)
    soft_test_predictions.append(model1.predict(test_inputs, batch_size=2))

In [ ]:
len(soft_test_predictions)

In [ ]:
soft_test_predictions[0].shape

In [ ]:
soft_test_predictions = np.mean(soft_test_predictions, axis=0)

In [ ]:
soft_test_predictions.shape

In [ ]:
hard_target_columns  = [
'question_asker_intent_understanding',
#  'question_body_critical',
#  'question_conversational',
#  'question_expect_short_answer',
#  'question_fact_seeking',
#  'question_has_commonly_accepted_answer',
#  'question_interestingness_others',
#  'question_interestingness_self',
#  'question_multi_intent',
'question_not_really_a_question',
#  'question_opinion_seeking',
#  'question_type_choice',
'question_type_compare',
'question_type_consequence',
#  'question_type_definition',
#  'question_type_entity',
#  'question_type_instructions',
#  'question_type_procedure',
#  'question_type_reason_explanation',
# 'question_type_spelling',
#  'question_well_written',
'answer_helpful',
#  'answer_level_of_information',
'answer_plausible',
'answer_relevance',
'answer_satisfaction',
#  'answer_type_instructions',
#  'answer_type_procedure',
#  'answer_type_reason_explanation',
#  'answer_well_written'
 ]

hard_test_predictions = []

for i in range(1,5):
    print(i)
    model_path = f'{model_weights_path}hard-bert-{i}.h5'
    model1 = create_model_hard(BERT_PATH)
    model1.load_weights(model_path)
    hard_test_predictions.append(model1.predict(test_inputs, batch_size=2))

In [ ]:
hard_test_predictions[0].shape

In [ ]:
hard_test_predictions = np.mean(hard_test_predictions, axis=0)

In [ ]:
hard_test_predictions.shape

In [ ]:
sample_submission[soft_target_columns] = soft_test_predictions

In [ ]:
sample_submission[hard_target_columns] = hard_test_predictions

In [ ]:
def question_type_spelling_hard(test):
    if test["category_type_english"] == 1 or test["category_type_ell"] == 1:
        
        if test["Tags_pronunciation"] == 1:
            return 0.666667
        elif test["Tags_spelling"] == 1:
            return 0.666667
        else:
            return 0.555555
    else:
        return 0.00000

In [ ]:
sample_submission["question_type_spelling"] = test_df.apply(question_type_spelling_hard, 1)

In [ ]:
def question_type_compare_hard(text):
    if text == np.nan:
        return 0.00000
    else:
        text = str(text)
        ls = text.split(" ")
        if "vs" in ls:
            return 1.000000
        elif ("between" or "difference") in ls:
            return 0.666667
        elif ("means" or "better") in ls:
            return 0.333333
        else:
            return 0.000000

In [ ]:
#sample_submission["question_type_compare"] = test_df.question_title_clean2.apply(question_type_compare_hard)

In [ ]:
sample_submission.nunique(axis=0)

In [ ]:
sample_submission.isna().sum().sum()

In [ ]:
sample_submission.fillna(0.5, inplace=True)

In [ ]:
sample_submission.iloc[:,1:].max().max()

In [ ]:
sample_submission.iloc[:,1:].min().min()

In [ ]:
#sample_submission.to_csv('submission.csv', index=False)

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv', index=False)

In [ ]:
TARGET_COLUMNS = target_columns

In [ ]:
TARGET_COLUMNS

In [ ]:
"""

from sklearn.preprocessing import MinMaxScaler
    
def postprocessing(oof_df):
   
    scaler = MinMaxScaler()
    
    # type 1 column [0, 0.333333, 0.5, 0.666667, 1]
    # type 2 column [0, 0.333333, 0.666667]
    # type 3 column [0.333333, 0.444444, 0.5, 0.555556, 0.666667, 0.777778, 0.8333333, 0.888889, 1]
    # type 4 column [0.200000, 0.266667, 0.300000, 0.333333, 0.400000, \
    # 0.466667, 0.5, 0.533333, 0.600000, 0.666667, 0.700000, \
    # 0.733333, 0.800000, 0.866667, 0.900000, 0.933333, 1]
    
    # comment some columns based on oof result
    
    ################################################# handle type 1 columns
    type_one_column_list = [
       'question_conversational', \
       'question_has_commonly_accepted_answer', \
       'question_not_really_a_question', \
       'question_type_choice', \
       'question_type_compare', \
       'question_type_consequence', \
       'question_type_definition', \
       'question_type_entity', \
       'question_type_instructions', 
    ]
    
    oof_df[type_one_column_list] = scaler.fit_transform(oof_df[type_one_column_list])
    
    tmp = oof_df.copy(deep=True)
    
    for column in type_one_column_list:
        
        oof_df.loc[tmp[column] <= 0.16667, column] = 0
        oof_df.loc[(tmp[column] > 0.16667) & (tmp[column] <= 0.41667), column] = 0.333333
        oof_df.loc[(tmp[column] > 0.41667) & (tmp[column] <= 0.58333), column] = 0.500000
        oof_df.loc[(tmp[column] > 0.58333) & (tmp[column] <= 0.73333), column] = 0.666667
        oof_df.loc[(tmp[column] > 0.73333), column] = 1
    
    
    
    ################################################# handle type 2 columns      
#     type_two_column_list = [
#         'question_type_spelling'
#     ]
    
#     for column in type_two_column_list:
#         if sum(tmp[column] > 0.15)>0:
#             oof_df.loc[tmp[column] <= 0.15, column] = 0
#             oof_df.loc[(tmp[column] > 0.15) & (tmp[column] <= 0.45), column] = 0.333333
#             oof_df.loc[(tmp[column] > 0.45), column] = 0.666667
#         else:
#             t1 = max(int(len(tmp[column])*0.0013),2)
#             t2 = max(int(len(tmp[column])*0.0008),1)
#             thred1 = sorted(list(tmp[column]))[-t1]
#             thred2 = sorted(list(tmp[column]))[-t2]
#             oof_df.loc[tmp[column] <= thred1, column] = 0
#             oof_df.loc[(tmp[column] > thred1) & (tmp[column] <= thred2), column] = 0.333333
#             oof_df.loc[(tmp[column] > thred2), column] = 0.666667
    
    
    
    ################################################# handle type 3 columns      
    type_three_column_list = [
       'question_interestingness_self', 
    ]
    scaler = MinMaxScaler(feature_range=(0, 1))
    oof_df[type_three_column_list] = scaler.fit_transform(oof_df[type_three_column_list])
    tmp[type_three_column_list] = scaler.fit_transform(tmp[type_three_column_list])
    
    for column in type_three_column_list:
        oof_df.loc[tmp[column] <= 0.385, column] = 0.333333
        oof_df.loc[(tmp[column] > 0.385) & (tmp[column] <= 0.47), column] = 0.444444
        oof_df.loc[(tmp[column] > 0.47) & (tmp[column] <= 0.525), column] = 0.5
        oof_df.loc[(tmp[column] > 0.525) & (tmp[column] <= 0.605), column] = 0.555556
        oof_df.loc[(tmp[column] > 0.605) & (tmp[column] <= 0.715), column] = 0.666667
        oof_df.loc[(tmp[column] > 0.715) & (tmp[column] <= 0.8), column] = 0.833333
        oof_df.loc[(tmp[column] > 0.8) & (tmp[column] <= 0.94), column] = 0.888889
        oof_df.loc[(tmp[column] > 0.94), column] = 1
        
        
        
    ################################################# handle type 4 columns      
    type_four_column_list = [
        'answer_satisfaction'
    ]
    scaler = MinMaxScaler(feature_range=(0.2, 1))
    oof_df[type_four_column_list] = scaler.fit_transform(oof_df[type_four_column_list])
    tmp[type_four_column_list] = scaler.fit_transform(tmp[type_four_column_list])
    
    for column in type_four_column_list:
        
        oof_df.loc[tmp[column] <= 0.233, column] = 0.200000
        oof_df.loc[(tmp[column] > 0.233) & (tmp[column] <= 0.283), column] = 0.266667
        oof_df.loc[(tmp[column] > 0.283) & (tmp[column] <= 0.315), column] = 0.300000
        oof_df.loc[(tmp[column] > 0.315) & (tmp[column] <= 0.365), column] = 0.333333
        oof_df.loc[(tmp[column] > 0.365) & (tmp[column] <= 0.433), column] = 0.400000
        oof_df.loc[(tmp[column] > 0.433) & (tmp[column] <= 0.483), column] = 0.466667
        oof_df.loc[(tmp[column] > 0.483) & (tmp[column] <= 0.517), column] = 0.500000
        oof_df.loc[(tmp[column] > 0.517) & (tmp[column] <= 0.567), column] = 0.533333
        oof_df.loc[(tmp[column] > 0.567) & (tmp[column] <= 0.633), column] = 0.600000
        oof_df.loc[(tmp[column] > 0.633) & (tmp[column] <= 0.683), column] = 0.666667
        oof_df.loc[(tmp[column] > 0.683) & (tmp[column] <= 0.715), column] = 0.700000
        oof_df.loc[(tmp[column] > 0.715) & (tmp[column] <= 0.767), column] = 0.733333
        oof_df.loc[(tmp[column] > 0.767) & (tmp[column] <= 0.833), column] = 0.800000
        oof_df.loc[(tmp[column] > 0.883) & (tmp[column] <= 0.915), column] = 0.900000
        oof_df.loc[(tmp[column] > 0.915) & (tmp[column] <= 0.967), column] = 0.933333
        oof_df.loc[(tmp[column] > 0.967), column] = 1
    
    
    ################################################# round to i / 90 (i from 0 to 90)
    oof_values = oof_df[TARGET_COLUMNS].values
    DEGREE = len(oof_df)//45*9
#     if degree:
#         DEGREE = degree
#     DEGREE = 90
    oof_values = np.around(oof_values * DEGREE) / DEGREE  ### 90 To be changed
    oof_df[TARGET_COLUMNS] = oof_values
    
    return oof_df
"""

In [ ]:
#sample_submission_post = postprocessing(sample_submission)

In [ ]:
#sample_submission_post.shape

In [ ]:
#for column in TARGET_COLUMNS:
#    print(sample_submission_post[column].value_counts())

In [ ]:
#sample_submission_post

In [ ]:
#sample_submission_post

In [ ]:
#sample_submission_post.to_csv('submission.csv', index=False)